<a href="https://colab.research.google.com/github/s-zeidi/Natural-language-inference-NLI/blob/main/code_03072025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Initial settings

In [ ]:
# !pip install sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
import numpy
from sklearn.metrics import f1_score,precision_score,recall_score
import json
import os
import torch
import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd



# New section

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
#file_path='./data/Complete_dataset/'
file_path='/content/drive/MyDrive/sepi_master_thesis/Complete_dataset/'


# 2. Dataset

In [ ]:
# !unzip /content/drive/MyDrive/sepi_master_thesis/NLI/Task-2-SemEval-2024/training_data.zip

In [ ]:
# Dev set #200
dev_path = file_path + 'dev.json'

with open(dev_path) as json_file:
    dev = json.load(json_file)


# Example instance
print(dev[list(dev.keys())[1]])

In [ ]:
# train set #1700
train_path = file_path + 'train.json'
with open(train_path) as json_file:
    train = json.load(json_file)

In [ ]:
# test set #500
test_path = file_path + 'test.json'
with open(test_path) as json_file:
    test = json.load(json_file)

In [ ]:
print(len(dev))
print(len(test))
print(len(train))


In [ ]:
train

# Functions

In [ ]:
#preparing  Qwen fintuning format d
import pandas as pd
import json
import os

# === Load CSV ===
csv_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/test_table.csv"
df = pd.read_csv(csv_path)

# === Filter for correct reasoning only ===
#df = df[df["Reasoning_status"].str.strip().str.lower() == "correct"]

# === Format D Output List ===
format_d_list = []

# === Process Each Example ===
for _, row in df.iterrows():
    primary = str(row.get("primary_section", "")).strip()
    secondary = row.get("secondary_section", "")
    if pd.isna(secondary) or str(secondary).strip().upper() == "N/A":
        secondary = ""
    else:
        secondary = str(secondary).strip()

    statement = str(row.get("statements", "")).strip()
    label = str(row.get("label", "")).strip()
    reasoning = str(row.get("Reasoning", "")).strip()

    # === Clean reasoning ===
    if reasoning.startswith("</think>"):
        reasoning = reasoning.replace("</think>", "", 1).strip()

    # === Input Prompt ===
    input_lines = [
        "You are a clinical reasoning assistant.",
        "Please provide exactly 4 clear and concise reasoning steps that justify whether the statement should be labeled as Entailment or Contradiction, based on the trial information.",
        "Use only evidence from the trial(s) — avoid assumptions or external knowledge.",
        "Please follow this format strictly:",
        "Step-by-step Explanation:",
        "Step 1: ...",
        "Step 2: ...",
        "Step 3: ...",
        "Step 4: ...",
        "Final Conclusion: The label (Entailment or Contradiction) is correct because ...",
        "Label: <Entailment or Contradiction>.",
        f"\nPrimary Trial: {primary}"
    ]
    if secondary:
        input_lines.append(f"\nSecondary Trial: {secondary}")
    input_lines.append(f"\nStatement: {statement}")

    input_text = "\n".join(input_lines)
    #output_text = f"\n{reasoning}\nLabel: {label}"
    output_text = f"{label}"

    format_d_list.append({
        #"instruction":"Based on the clinical trial information and the statement, predict whether the label is Entailment or Contradiction. Provide four clear reasoning steps, followed by a final conclusion and the predicted label in the specified format.",
        "input": input_text,
        #"output": output_text,
        "label": output_text
    })

# === Save to JSON ===
#output_path = "/content/drive/MyDrive/sepi_master_thesis/qwen_format_d/dev.json"  # ⬅️ Change to your output path
#with open(output_path, "w", encoding="utf-8") as f:
    #json.dump(format_d_list, f, indent=2, ensure_ascii=False)

#print(f"✅ Saved {len(format_d_list)} examples to '{output_path}'")

# === Save as CSV ===
csv_output_path = "/content/drive/MyDrive/sepi_master_thesis/qwen_format_d/test.csv"
csv_df = pd.DataFrame(format_d_list)
csv_df.to_csv(csv_output_path, index=False, encoding="utf-8")

print(f"✅ Also saved {len(format_d_list)} examples to '{csv_output_path}'")

In [ ]:
#format_d_list[2]['input'].split('\n')
#format_d_list[100]['output'].split('\n')
#format_d_list[1]['instruction'].split('\n')
csv_df['input'].iloc[0].split('\n')

In [ ]:
#preparing  Qwen fintuning format
import pandas as pd
import json

# === File Paths ===
#csv_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/dev.csv"
csv_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/test_table.csv"
output_dir = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/"

# === Load CSV ===
df = pd.read_csv(csv_path)

# === Output Lists ===
format_a_list = []  # Statement in input
format_b_list = []  # Statement in instruction

# === Process Each Example ===
for _, row in df.iterrows():
    primary = str(row.get("primary_section", "")).strip()
    #secondary = str(row.get("secondary_section", "")).strip()
    secondary = row.get("secondary_section", "")
    if pd.isna(secondary) or str(secondary).strip().upper() == "N/A":
        secondary = ""
    else:
        secondary = str(secondary).strip()

    statement = str(row.get("statements", "")).strip()
    #label = str(row.get("label", "")).strip().lower()

    if not primary or not statement: #or not label:
        continue  # skip incomplete rows

    # === Format A: Statement in INPUT ===
    input_lines_a = [f"Primary Trial: {primary}"]
    if secondary:
        input_lines_a.append(f"Secondary Trial: {secondary}")
    input_lines_a.append(f"Statement: {statement}")
    input_text_a = "\n".join(input_lines_a)

    entry_a = {
        "instruction": "Do the clinical trial descriptions imply the given statement? Answer with only one of the options (Entailment or Contradiction):",
        "input": input_text_a,
        #"output": label
    }
    format_a_list.append(entry_a)

    # === Format B: Statement in INSTRUCTION ===
    input_lines_b = [f"Primary Trial: {primary}"]
    if secondary:
        input_lines_b.append(f"Secondary Trial: {secondary}")
    input_text_b = "\n".join(input_lines_b)

    instruction_b = (
        f"Question: Do the clinical trial descriptions imply that {statement}? "
        f"Answer with only one of the options (Entailment or Contradiction):"
    )

    entry_b = {
        "instruction": instruction_b,
        "input": input_text_b,
        #"output": label
    }
    format_b_list.append(entry_b)

# === Save Both JSON Files ===
file_a = os.path.join(output_dir, "format_a_statement_in_input.json")
file_b = os.path.join(output_dir, "format_b_statement_in_instruction.json")

with open(file_a, "w", encoding="utf-8") as f1:
    json.dump(format_a_list, f1, indent=2, ensure_ascii=False)

with open(file_b, "w", encoding="utf-8") as f2:
    json.dump(format_b_list, f2, indent=2, ensure_ascii=False)

print(f"✅ Saved {len(format_a_list)} examples to '{file_a}'")
print(f"✅ Saved {len(format_b_list)} examples to '{file_b}'")


In [ ]:
df

In [ ]:
format_a_list[1696]['input'].split('\n')


In [ ]:
def get_input_text(premise, hypothesis):
    input_text = f"{premise} \n Question: Does this imply that {hypothesis}? Answer with only one of the options (Entailment or Contradiction):"
    return input_text

In [ ]:
def get_input_text_2shot(premise, hypothesis): # this is for mistral(optimized)
    options_prefix = "OPTIONS:\n- "
    separator = "\n- "
    options_ = options_prefix + f"{separator}".join(["Entailment", "Contradiction"])

    # Two-shot examples
    example_1="""Primary trial evidence are Inclusion Criteria: Age 52-75 years old; Identification as Latina/Hispanic/Chicana female;
    Residence in Pilsen, Little Village, East Side or South Chicago; No history of health volunteerism; No history of breast cancer; and Lack of
    a mammogram within the last two yearsExclusion Criteria: Not meeting all inclusion criteria; Women will be excluded if they participated in
    formative focus groups Question: Does this imply that Patients eligible for the primary trial must live in the USA?
    Answer with only one of the options (Entailment or Contradiction): Entailment"""

    example_2="""Primary trial evidence are Adverse Events 1: Total: 10/71 (14.08%) ATRIAL FIBRILLATION 1/71 (1.41%) CARDIAC TAMPONADE 1/71 (1.41%)
    PERICARDIAL EFFUSION 1/71 (1.41%) SUPRAVENTRICULAR TACHYCARDIA 1/71 (1.41%) DIARRHOEA 1/71 (1.41%) NAUSEA 1/71 (1.41%) VOMITING 1/71 (1.41%)
    CHEST PAIN 1/71 (1.41%) PNEUMONIA 1/71 (1.41%) MALIGNANT PLEURAL EFFUSION 1/71 (1.41%) HEPATIC ENCEPHALOPATHY 1/71 (1.41%)
    Question: Does this imply that There were only 3 adverse events in the primary trial which occurred more than twice?
    Answer with only one of the options (Entailment or Contradiction): Contradiction"""




   # Introduction text with examples
    example_intro = "Here are my first examples: \n "
    example_intro2 = "Here are my second examples: \n "

    # Example text (Two-shot)
    two_shot_text = (
        f"{example_intro}"
        f"Example 1: {example_1} \n "
        f"{example_intro2}"
        f"Example 2: {example_2} \n ")


    # For the actual input
    input_text = f"{premise} \n Question: Does this imply that {hypothesis}? Answer with only one of the options (Entailment or Contradiction):"

    # Combine two-shot examples with the actual input
    return two_shot_text + input_text

In [ ]:
def get_input_text2_2shot(premise, hypothesis):
    options_prefix = "OPTIONS:\n- "
    separator = "\n- "
    options_ = options_prefix + f"{separator}".join(["Entailment", "Contradiction"])

    example_1 = """Example 1:
    Clinical Trial Report:
    Primary Trial: Inclusion Criteria: Age 52-75 years old; Identification as Latina/Hispanic/Chicana female;
    Residence in Pilsen, Little Village, East Side or South Chicago; No history of health volunteerism; No history of breast cancer; and Lack of
    a mammogram within the last two years Exclusion Criteria: Not meeting all inclusion criteria; Women will be excluded if they participated in
    formative focus groups.

    Statement: Patients eligible for the primary trial must live in the USA.
    Label: Entailment"""

    example_2 = """Example 2:
    Clinical Trial Report:
    Primary Trial: Adverse Events 1: Total: 10/71 (14.08%) ATRIAL FIBRILLATION 1/71 (1.41%) CARDIAC TAMPONADE 1/71 (1.41%)
    PERICARDIAL EFFUSION 1/71 (1.41%) SUPRAVENTRICULAR TACHYCARDIA 1/71 (1.41%) DIARRHOEA 1/71 (1.41%) NAUSEA 1/71 (1.41%) VOMITING 1/71 (1.41%)
    CHEST PAIN 1/71 (1.41%) PNEUMONIA 1/71 (1.41%) MALIGNANT PLEURAL EFFUSION 1/71 (1.41%) HEPATIC ENCEPHALOPATHY 1/71 (1.41%).


    Statement: There were only 3 adverse events in the primary trial which occurred more than twice.
    Label: Contradiction"""

    example_intro = "Below are two examples:"
    example_intro2 = "Now, classify the following example using only 'ENTAILMENT' or 'CONTRADICTION'."

    two_shot_text = (
        f"{example_intro}\n\n"
        f"{example_1} \n\n"
        f"{example_2} \n\n"
        f"{example_intro2}\n\n"
    )

    input_text = (
        f"Clinical Trial Report:\n"
        f"{premise}\n"
        f"Statement: {hypothesis}\n"
        f"Label:('Entailment' or 'Contradiction')?"
        )

    return two_shot_text + input_text


In [ ]:
def get_input_text3(premise, hypothesis):
    options_prefix = "OPTIONS:\n- "
    separator = "\n- "
    options_ = options_prefix + f"{separator}".join(["Entailment", "Contradiction"])

    example_intro2 = "Classify the following example using only 'ENTAILMENT' or 'CONTRADICTION'."

    zero_shot_text = (
        f"{example_intro2}\n\n"
    )

    input_text = (
        f"Clinical Trial Report:\n"
        f"{premise}\n"
        f"Statement: {hypothesis}\n"
        f"Label:('Entailment' or 'Contradiction')?"
        )

    return zero_shot_text + input_text


In [ ]:
def get_input_cot(premise, hypothesis, label):
    input_text = (
        f"Clinical Trial Report:\n"
        f"{premise}\n"
        f"Statement: {hypothesis}\n"
        f"Label: {label}\n\n"
    )

    explanation_prompt = f"Explain step-by-step why '{label}' is the appropriate label given the trial(s) and the statement:"

    return input_text + explanation_prompt


In [ ]:
def get_input_cot4(primary_trial, hypothesis, label, secondary_trial=None):
    prompt = (
        "You are a clinical reasoning assistant.\n\n"
        f"Please provide exactly 4 clear and concise reasoning steps that explain why the label '{label}' "
        f"is appropriate given the trial information and the statement.\n\n"
        f"Use only evidence from the trial(s) — avoid assumptions or external knowledge.\n\n"
        f"Please follow this format strictly:\n\n"
        f"Step-by-step Explanation:\n"
        f"Step 1: ...\n"
        f"Step 2: ...\n"
        f"Step 3: ...\n"
        f"Step 4: ...\n\n"
        f"Final Conclusion: The label is correct because ..."
    )

    # Add the trial(s) and statement section
    prompt += "\n\nClinical Trial Report:\n"
    prompt += f"Primary Trial:\n{primary_trial.strip()}\n"
    if secondary_trial and len(secondary_trial.strip()) > 0:
        prompt += f"\nSecondary Trial:\n{secondary_trial.strip()}\n"
    prompt += f"\nStatement:\n{hypothesis.strip()}\n"

    return prompt


In [ ]:
def get_input_cot2(premise, hypothesis, label):
    input_text = (
        f"Clinical Trial Report:\n"
        f"{premise}\n"
        f"Statement: {hypothesis}\n"
        f"Label: {label}\n\n"
    )

    explanation_prompt = (
        f"Given the detailed trial(s) information and the statement above, articulate step-by-step (up to 5 steps) "
        f"why the label '{label}' accurately reflects the relationship between the statement and the trial data. "
        f"Focus on key evidence and use logical deductions to support each step of your reasoning. Ensure that each "
        f"step is factually based on the provided information and contributes directly to justifying the assigned label."
    )

    return input_text + explanation_prompt


In [ ]:
def get_input_cot3(premise, hypothesis, label):
    input_text = (
        f"Clinical Trial Report:\n"
        f"{premise}\n"
        f"Statement: {hypothesis}\n"
        f"Label: {label}\n\n"
    )

    explanation_prompt = (
        f"Please provide up to 5 clear and concise reasoning steps that explain why the label '{label}' is appropriate given the trial information and the statement. Each step should directly contribute to establishing whether there is a contradiction or entailment between the trials' data and the statement."
    )

    return input_text + explanation_prompt



In [ ]:
import pandas as pd
file_path_input = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/combined_test(text,statement,lables).csv"
file_path_output="/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/test-zero-shot.csv"
df=pd.read_csv(file_path_input)
def create_cot_data(input_csv, output_csv):

    df = pd.read_csv(input_csv)
    if not {'text', 'statements', 'label'}.issubset(df.columns):
        raise ValueError("Dataset must contain 'text', 'statements', and 'label' columns.")

    df['text'] = df.apply(lambda row: get_input_text(row['text'], row['statements']), axis=1)
    df = df[['text', 'label']]
    df.to_csv(output_csv, index=False)
    print(f"2-shot test dataset saved as: {output_csv}")

create_cot_data(file_path_input,file_path_output)

In [ ]:
import pandas as pd
file_path_input = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/train.csv"
file_path_output="/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/train-finetune.csv"
df=pd.read_csv(file_path_input)
def create_2shot_test_data(input_csv, output_csv):

    df = pd.read_csv(input_csv)
    if not {'text', 'statements', 'label'}.issubset(df.columns):
        raise ValueError("Dataset must contain 'text', 'statements', and 'label' columns.")

    df['text'] = df.apply(lambda row: get_input_text(row['text'], row['statements']), axis=1)
    df = df[['text', 'label']]
    df.to_csv(output_csv, index=False)
    print(f"2-shot test dataset saved as: {output_csv}")

create_2shot_test_data(file_path_input,file_path_output)

In [ ]:
import pandas as pd

def process_samples(file_path, get_input_cot):
    """
    Process the samples from an Excel file, create input text based on primary and secondary trial evidence,
    and append to a list of dictionaries containing 'text' and 'label'.

    Parameters:
    - file_path: Path to the Excel file
    - get_input_text: Function to prepare input text

    Returns:
    - samples: List of dictionaries with 'text' and 'label'
    """

    # Load the Excel file into a DataFrame
    #data_expanded = pd.read_excel(file_path)
    data_expanded = pd.read_csv(file_path)
    samples = []

    for _, sample in data_expanded.iterrows():  # Iterate over rows
        primary_evidence = "".join(str(sample['primary_section']))
        text = f"Primary trial evidence are {primary_evidence}"

        # Check if secondary evidence is not null
        if pd.notna(sample["secondary_section"]):
            text = f"{text} Secondary trial evidence are {sample['secondary_section']}"

        # Generate input text using the provided get_input_text function
        #input_text = get_input_text(text, sample['statements'])
        input_text = get_input_cot4(text, sample['statements'],sample['label'])
        # Append the sample to the list as a dictionary
        temp = {"text": input_text, "label": sample['label']}
        samples.append(temp)

    return samples


In [ ]:
#file_path_input = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/train.csv"
file_path_input = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/dev.csv"

file_path_output="/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/dev_cot_input4.csv"




samples_1shot = process_samples(file_path_input, get_input_cot4)

In [ ]:
sample = pd.DataFrame(samples_1shot)

In [ ]:
sample.iloc[0]
sample.iloc[0]['text'].split('\n')


In [ ]:
sample

In [ ]:
sample.to_csv(file_path_output, index=False)



In [ ]:
def clean_pred(preds):
    cleaned_pred = []
    for pred in preds:
        # Extract the string from the list (assuming each sentence is a list with one element)
        if isinstance(pred, list):
            pred = pred[0]

        # Strip unwanted tokens and spaces
        pred = pred.replace("<pad>", "").replace("</s>", "").strip()

        # Check for "Contradiction" or "Entailment"
        if "Yes" in pred or "Entailment" in pred:
            cleaned_pred.append("Entailment")
        elif "No" in pred or "Contradiction" in pred:
            cleaned_pred.append("Contradiction")

    return cleaned_pred

In [ ]:
data=dev
uuid_list = list(data.keys())
statements = []
gold_dev_primary_evidence = []
gold_dev_secondary_evidence = []
for i in range(len(uuid_list)):
  #Retrieve all statements from the development set
  statements.append(data[uuid_list[i]]["Statement"])

In [ ]:
data=dev

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

Results = {}
df_data = []

# Loop through the uuid_list
for i in range(len(uuid_list)):
    primary_ctr_path = os.path.join(file_path, 'CT json', data[uuid_list[i]]["Primary_id"] + ".json")

    # Load the primary trial data
    with open(primary_ctr_path) as json_file:
        primary_ctr = json.load(json_file)

    # Retrieve the full section from the primary trial
    primary_section = primary_ctr[data[uuid_list[i]]["Section_id"]]

    # Convert the primary section entries to a matrix of TF-IDF features
    vectorizer = TfidfVectorizer().fit(primary_section)
    X_s = vectorizer.transform([statements[i]])
    X_p = vectorizer.transform(primary_section)

    # Compute the cosine similarity between the primary section entries and the statement
    primary_scores = cosine_distances(X_s, X_p)

    # Initialize values for secondary section and prediction
    secondary_section = None
    Prediction = None

    # If there is a comparison trial, repeat for the secondary trial
    if data[uuid_list[i]]["Type"] == "Comparison":
        secondary_ctr_path = os.path.join(file_path, 'CT json', data[uuid_list[i]]["Secondary_id"] + ".json")

        with open(secondary_ctr_path) as json_file:
            secondary_ctr = json.load(json_file)

        secondary_section = secondary_ctr[data[uuid_list[i]]["Section_id"]]

        # Apply TF-IDF for secondary section
        vectorizer = TfidfVectorizer().fit(secondary_section)
        X_s = vectorizer.transform([statements[i]])
        X_p = vectorizer.transform(secondary_section)
        secondary_scores = cosine_distances(X_s, X_p)

        # Combine and average the cosine distances from both trials
        combined_scores = []
        combined_scores.extend(secondary_scores[0])
        combined_scores.extend(primary_scores[0])
        score = np.average(combined_scores)

        # Determine prediction based on the cosine distance
        if score > 0.9:
            Prediction = "Contradiction"
        else:
            Prediction = "Entailment"
    else:
        # Use only the primary trial for prediction
        score = np.average(primary_scores)

        if score > 0.9:
            Prediction = "Contradiction"
        else:
            Prediction = "Entailment"

        secondary_section = "N/A"  # If no secondary trial is present

    # Store prediction in Results dictionary
    Results[str(uuid_list[i])] = {"Prediction": Prediction}

    # Append data for the dataframe
    df_data.append({
        "primary_section": primary_section,
        "secondary_section": secondary_section,
        "statements": statements[i],
        "label": data[uuid_list[i]]["Label"],
        "TF_IDF_prediction": Prediction
    })

# Create a DataFrame from the collected data
df = pd.DataFrame(df_data)

# Save the DataFrame to an Excel file
# output_file = 'methods_output_test_data.xlsx'  # You can specify a desired path
# output_file = 'train_data_new_format.xlsx'  # You can specify a desired path
# output_file = 'test_data_new_format.xlsx'  # You can specify a desired path
output_file = '/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/dev.csv'
df.to_csv(output_file, index=False)

# df.to_excel(output_file, index=False)

print(f"DataFrame saved to {output_file}")


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Use the small Flan-T5 model
# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", cache_dir="/mnt/data/huggingface_cache", device_map="auto")

# Use the xl Flan-T5 model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", cache_dir="./data/huggingface_cache" )


In [ ]:
# Define or import your get_input_text function
def get_input_text(premise, hypothesis):
    options_prefix = "OPTIONS:\n- "
    separator = "\n- "
    options_ = options_prefix + f"{separator}".join(["Entailment","Contradiction"])
    return f"{premise} \n Question: Does this imply that {hypothesis} Choose one: [entailment, contradiction].? "

# Call the process_samples function
samples = process_samples("./test_data_new_format.xlsx", get_input_text)

# Output the samples list
#print(samples)



In [ ]:
save_path = './data/zero_shot_test_data.json'
with open(save_path, 'w') as f:
    json.dump(samples, f)

# zero_shot


In [ ]:
labels = []
pred = []
with torch.inference_mode():
    for sample in tqdm.tqdm(samples):
        labels.append(sample["label"])
        input_ids = tokenizer(sample["text"], return_tensors="pt")
        # print(input_ids)

        outputs = model.generate(**input_ids)
        # pred.append(tokenizer.decode(outputs[0]))
        pred.append(tokenizer.batch_decode(outputs))

In [ ]:

# # Load the Excel file into a DataFrame
# d_path="methods_output_test_data.xlsx"
# df = pd.read_excel(d_path)

# # Add the new column to the DataFrame
# # df['zero_shot_base'] = pred
# df['zero_shot_base_clean_pred'] = clean_pred(pred)

# # Save the updated DataFrame back to an Excel file
# df.to_excel(d_path, index=False)

# df

In [ ]:
# # Compute metrics with different averaging methods ???????????????small ????
# metrics_micro = compute_metrics(labels, pred, averaging="micro")
# metrics_macro = compute_metrics(labels, pred, averaging="macro")
# metrics_weighted = compute_metrics(labels, pred, averaging="weighted")
# #save_metrics_to_csv('FLAN-T5-zero-shot)', metrics_micro, metrics_macro, metrics_weighted, result_path)
# # Print results
# print("Micro averaging:", metrics_micro)
# print("Macro averaging:", metrics_macro)
# print("Weighted averaging:", metrics_weighted)

In [ ]:
result_table = evaluate.evaluate_all_methods('./methods_output_test_data.xlsx','./results_test_data.xlsx')
result_table

#  FLAN-T5_xl

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Use the small Flan-T5 model
# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", cache_dir="/mnt/data/huggingface_cache", device_map="auto")

# Use the xl Flan-T5 model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", cache_dir="./data/huggingface_cache" )

In [ ]:
samples

In [ ]:
import tqdm
import torch

model = model.to('cuda')

labels = []
pred = []
samples = data  # Assuming this is a pandas DataFrame
with torch.inference_mode():
    for _, item in tqdm.tqdm(samples.iterrows(), total=len(samples), desc="Processing Samples"):
        text = item["text"]  # Access the "text" column
        inputs = tokenizer(text, return_tensors="pt", max_length=2048, truncation=True).to('cuda')
        outputs = model.generate(**inputs, max_length=10)
        pred.append(tokenizer.batch_decode(outputs))


In [ ]:
data['text'].iloc[233]

In [ ]:

model = model.to('cuda')

input_text="""Here are my first examples:
 Example 1: Primary trial evidence are Inclusion Criteria: Age 52-75 years old; Identification as Latina/Hispanic/Chicana female;
    Residence in Pilsen, Little Village, East Side or South Chicago; No history of health volunteerism; No history of breast cancer; and Lack of
    a mammogram within the last two yearsExclusion Criteria: Not meeting all inclusion criteria; Women will be excluded if they participated in
    formative focus groups Question: Does this imply that Patients eligible for the primary trial must live in the USA?
    Answer with only one of the options (Entailment or Contradiction): Entailment
 Here are my second examples:
 Example 2: Primary trial evidence are Adverse Events 1: Total: 10/71 (14.08%) ATRIAL FIBRILLATION 1/71 (1.41%) CARDIAC TAMPONADE 1/71 (1.41%)
    PERICARDIAL EFFUSION 1/71 (1.41%) SUPRAVENTRICULAR TACHYCARDIA 1/71 (1.41%) DIARRHOEA 1/71 (1.41%) NAUSEA 1/71 (1.41%) VOMITING 1/71 (1.41%)
    CHEST PAIN 1/71 (1.41%) PNEUMONIA 1/71 (1.41%) MALIGNANT PLEURAL EFFUSION 1/71 (1.41%) HEPATIC ENCEPHALOPATHY 1/71 (1.41%)
    Question: Does this imply that There were only 3 adverse events in the primary trial which occurred more than twice?
    Answer with only one of the options (Entailment or Contradiction):
    Contradiction
 Primary trial evidence are: ['Outcome Measurement: ', '  Progression Free Survival (PFS)', '  PFS is defined as the time from randomization to the earlier of disease progression or death due to any cause. Disease progression is defined using Response Evaluation Criteria In Solid Tumors Criteria (RECIST v1.1), as a 20% increase in the sum of the longest diameter of target lesions, or progression in a non-target lesion, or the appearance of new lesions. The primary analysis of PFS was based on PFS events determined retrospectively by the central independent review committee, blinded to treatment assignment and investigator assessments according to RECIST 1.1 criteria.', '  Time frame: Evaluated every 6 - 9 weeks following treatment initiation', 'Results 1: ', '  Arm/Group Title: Capecitabine', '  Arm/Group Description: Capecitabine administered on Days 1 through 14 of each 21 day cycle until disease progression, discontinuation due to toxicity, withdrawal of consent, or end of study.', '  Overall Number of Participants Analyzed: 109', '  Median (95% Confidence Interval)', '  Unit of Measure: months  2.8        (1.6 to 3.2)', 'Results 2: ', '  Arm/Group Title: CDX-011', '  Arm/Group Description: CDX-011 administered on Day 1 of each 21 day cycle until disease progression, discontinuation due to toxicity, withdrawal of consent, or end of study.', '  Overall Number of Participants Analyzed: 218', '  Median (95% Confidence Interval)', '  Unit of Measure: months  2.9        (2.8 to 3.5)']
 Question: Does this imply that The patient with the longest PFS in the primary trial was in cohort 1, and he survived 3.5 months without disease progression or death ? Answer with only one of the options (Entailment or Contradiction):"""
inputs = tokenizer(input_text, return_tensors="pt", max_length=2048, truncation=True).to('cuda')
outputs = model.generate(**inputs, max_length=199)

# Decode and print the result
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Answer:", result)

In [ ]:
model = model.to('cuda')

input_text = """Primary trial evidence are: ['Inclusion Criteria:', '  Female patients, >/= 18 years of age', '  Histologically confirmed HER2-negative breast cancer', '  Disease progression during or following first-line treatment with Avastin and chemotherapy for locally recurrent or metastatic breast cancer', '  Avastin treatment in first-line setting must have been a minimum of 4 cycles (15 mg/kg) or 6 cycles (10 mg/kg) in combination with chemotherapy', '  ECOG performance status 0-2', '  At least 28 days since prior radiation therapy or surgery and recovery from treatment', 'Exclusion Criteria:', '  Anti-angiogenic therapy or anti-vascular endothelial growth factors other than Avastin for first-line treatment', '  Active malignancy other than superficial basal cell and superficial squamous cell carcinoma of the skin, or in situ carcinoma of the cervix or breast within the last 5 years', '  Inadequate renal function', '  Clinically relevant cardio-vascular disease', '  Known CNS disease except for treated brain metastases', '  Chronic daily treatment with high-dose aspirin (>325 mg/day) or clopidogrel (>75 mg/day)', '  Pregnant or lactating women']
Secondary trial evidence are: ['Inclusion Criteria:', '  Female or male  18 years of age.', '  Newly diagnosed ER-positive, HER2-negative breast cancer. ER-positive is defined as  1% immunohistochemical (IHC) staining of any intensity. HER2 test result is negative if a single test (or both tests) performed show:', '  IHC 1+ or 0', '  In situ hybridization negative based on:', '  Single-probe average HER2 copy number < 4.0 signals/cell', '  Dual-probe HER2/CEP17 ratio < 2 with an average HER2 copy number < 4.0 signals/cell.', '  Patients with stage II-III breast cancer are eligible if they are deemed appropriate for neoadjuvant endocrine therapy by the referring or treating medical oncologist. Patients with stage I disease are eligible if they are deemed borderline candidates for breast conservation and the treating surgeon recommends preoperative therapy to increase the chances of breast conservation.', '  Eastern Cooperative Oncology Group performance status and/or other performance status of  1.', '  Female patients who:', '  Are postmenopausal for at least 1 year before the screening visit, OR', '  Are surgically sterile, OR', '  If they are of childbearing potential, agree to practice 1 effective method of contraception and 1 additional effective (barrier) method, at the same time, from the time of signing the ICF through 90 days (or longer, as mandated by local labeling [e.g., United Surgical Partners International, summary of product characteristics, etc.] after the last dose of the study drugs, OR', '  Agree to practice true abstinence, when this is in line with the preferred and usual lifestyle of the patient (periodic abstinence [e.g., calendar, ovulation, symptothermal, postovulation methods] and withdrawal, spermicides only, and lactational amenorrhea are not acceptable methods of contraception. Female and male condom should not be used together).', '  Male patients, even if surgically sterilized (i.e., status post-vasectomy), who:', '  Agree to practice highly effective barrier contraception during the entire study treatment period and through 120 days after the last dose of the study drugs, OR', '  Agree to practice true abstinence, when this is in line with the preferred and usual lifestyle of the patient', '  Agree not to donate sperm during the course of this study or within 120 days after receiving their last dose of the study drugs.', '  Screening clinical laboratory values as specified below:', '  Bone marrow reserve consistent with: absolute neutrophil count  1.5 x 109/L, platelet count  100 x 109/L, and hemoglobin  9 g/dL (without transfusion) within 1 week preceding the administration of the study drugs;', "  Hepatic status: Serum total bilirubin  1 x upper limit of normal (ULN; in the case of known Gilbert's syndrome, a higher serum total bilirubin [< 1.5 x ULN] is allowed), aspartate aminotransferase and alanine aminotransferase  1.5 x ULN, and alkaline phosphatase  1.5 x ULN;", '  Renal status: Creatinine clearance 50 mL/min based on Cockcroft-Gault estimate or based on urine collection (12 or 24 hour);', '  Metabolic status: HbA1c < 7.0%, fasting serum glucose  130 mg/dL, and fasting triglycerides  300 mg/dL.', '  Ability to swallow oral medications.', '  Voluntary written consent must be given before performance of any study-related procedure not part of standard medical care, with the understanding that consent may be withdrawn by the patient at any time without prejudice to future medical care.', '  Negative serum pregnancy test within 7 days prior to the administration of the study drugs for female patients of childbearing potential.', '  Patient must be accessible for treatment and follow-up.', '  Patient must be willing to undergo breast biopsies as required by the study protocol.', 'Exclusion Criteria:', '  Any patient with metastatic disease.', "  Other clinically significant comorbidities, such as uncontrolled pulmonary disease, active central nervous system disease, active infection, or any other condition that could compromise the patient's participation in the study.", '  Known human immunodeficiency virus infection.', '  Known hepatitis B surface antigen-positive or known or suspected active hepatitis C infection.', "  Any serious medical or psychiatric illness that could, in the investigator's opinion, potentially interfere with the completion of the protocol-specified treatment.", '  Diagnosed or treated for another malignancy within 2 years before administration of the first dose of the study drugs or previously diagnosed with another malignancy and have any evidence of residual disease. Patients with non-melanoma skin cancer or carcinoma in situ of any type are not excluded if they have undergone complete resection.", '  Breastfeeding or pregnant.', '  Manifestations of malabsorption due to prior gastrointestinal surgery, gastrointestinal disease, or an unknown reason that may alter the absorption of TAK-228. Patients with enteric stomata are also excluded.', '  Treatment with any investigational products within 2 weeks before administration of the first dose of the study drugs.', '  Poorly controlled diabetes mellitus (defined as HbA1c > 7%). Patients with a history of transient glucose intolerance due to corticosteroid administration may be enrolled in the study if all other inclusion criteria and none of the other exclusion criteria are met.', '  History of any of the following within the last 6 months before administration of the first dose of the study drugs:', '  Ischemic myocardial event, including angina requiring therapy and artery revascularization procedures', '  Ischemic cerebrovascular event, including transient ischemic attack and artery revascularization procedures', '  Requirement for inotropic support (excluding digoxin) or serious (uncontrolled) cardiac arrhythmia (including atrial flutter/fibrillation, ventricular fibrillation, and ventricular tachycardia)', '  Placement of a pacemaker for control of rhythm', '  New York Heart Association Class III or IV heart failure', '  Pulmonary embolism', '  Significant active cardiovascular or pulmonary disease including:', '  Uncontrolled hypertension (i.e., systolic blood pressure > 180 mm Hg, diastolic blood pressure > 95 mm Hg). Use of antihypertensive agents to control hypertension before week 1, day 1 is allowed.', '  Pulmonary hypertension', '  Uncontrolled asthma or O2 saturation < 90% by arterial blood gas analysis or pulse oximetry on room air', '  Significant valvular disease, severe regurgitation, or stenosis by imaging independent of symptom control with medical intervention or history of valve replacement', '  Medically significant (symptomatic) bradycardia', '  History of arrhythmia requiring an implantable cardiac defibrillator', '  Baseline QTc prolongation (e.g., repeated demonstration of QTc interval > 480 milliseconds or history of congenital long QT syndrome or torsades de pointes)', '  Treatment with strong inhibitors and/or inducers of CYP3A4, CYP2C9, or CYP2C19 within 7 days preceding the first dose of the study drugs.', '  Patients receiving systemic corticosteroids (either IV or oral steroids, excluding inhalers or low-dose hormone replacement therapy) within 1 week before administration of the first dose of the study drugs.', '  Daily or chronic use of a proton pump inhibitor (PPI) and/or having taken a PPI within 7 days before receiving the first dose of the study drugs.', '  Patients unwilling or unable to comply with the study protocol.', '  Patients previously treated with hormonal therapy (tamoxifen, AI) or PI3K, AKT, dual PI3K/mTOR, TORC1/2, or mTORC1 inhibitors.', '  Patients who are currently being treated with cancer therapy (chemotherapy, radiation therapy, immunotherapy, or biologic therapy) other than the trial therapy.', '  Patients with hypersensitivity to mTOR inhibitors or tamoxifen.']
Question: does this imply that A 20 year old female patient with a Newly diagnosed ER-positive, HER2-negative breast cancer, could be eligible for the secondary trial and the primary trial
choose only one of the options (Entailment or Contradiction):
"""

# Truncate the input text to fit within the model's input size (2048 tokens max)
max_input_length = 2048
input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=max_input_length).to('cuda')

# Generate output from the model
with torch.inference_mode():
    outputs = model.generate(**input_ids)

# Decode the output
pred = tokenizer.batch_decode(outputs)

print("Prediction:", pred[0])  # Accessing the first (and only) prediction


In [ ]:
model = model.to('cuda')  # Move the model to GPU


In [ ]:
# Ensure model is on GPU
#skipped_rows = [7, 9, 24, 25, 26, 27, 35, 38, 41, 49, 50, 61, 65, 73, 74, 75, 78, 117, 120, 130, 139, 145, 146, 149, 163, 169, 175, 176, 220, 235, 246, 261, 265, 269, 277, 280, 284, 295, 296, 314, 319, 324, 329, 332, 355, 357, 360, 367, 374, 377, 404, 409, 414, 418, 421, 423, 427, 449, 466, 468, 469, 470, 472, 478, 482, 483, 490, 491]
skipped_rows = [24, 25, 26, 117, 145, 176, 220, 374, 414, 418]

model = model.to('cuda')

# Process each skipped row
for row_index in tqdm(skipped_rows, desc="Processing rows", unit="row"):
    try:
        # Get the text for the current row
        text = df.loc[row_index, 'text']

        # Prepare the input for the model and move to GPU
        inputs = tokenizer(text, return_tensors="pt", max_length=2048, truncation=True).to('cuda')

        # Generate the output (model will now use the same device as inputs)
        outputs = model.generate(**inputs, max_length=100)  # Adjust max_length as needed
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Update the prediction column with the result
        df.at[row_index, 'prediction'] = result

    except KeyError:
        print(f"Row {row_index} not found in the dataset.")
    except Exception as e:
        print(f"Error processing row {row_index}: {e}")


In [ ]:
from tqdm import tqdm
import pandas as pd
import torch

# Assume 'data' is your dataframe and contains a column 'text'
df = pd.DataFrame(data)

# List of important row numbers (replace with your list)
important_rows = [24, 25, 26, 78, 117, 145, 176, 220, 277, 374, 414, 418]# Example row indices

# Select the text from the important rows
samples = df.iloc[important_rows]["text"].tolist()

# Initialize list to store predictions
pred = []

# Use inference mode for efficient processing
with torch.inference_mode():
    for text in tqdm(samples, desc="Processing"):
        input_ids = tokenizer(text, return_tensors="pt", max_length=2048, truncation=True)
        outputs = model.generate(**input_ids,max_length=2048)
        pred.append(tokenizer.batch_decode(outputs, skip_special_tokens=True))

# The result is stored in 'pred' which contains the model's output for the selected rows


In [ ]:
# input_text ="""Primary trial evidence are: ['Adverse Events 1:', '  Total: 1/13 (7.69%)', '  Rapid disease progression  [1]1/13 (7.69%)', '  Increased pleural effusion  [2]0/13 (0.00%)', 'Adverse Events 2:', '  Total: 1/4 (25.00%)', '  Rapid disease progression  [1]0/4 (0.00%)', '  Increased pleural effusion  [2]1/4 (25.00%)']
#  Question: does this imply that 25% of patients in the primary trial suffer Increased pleural effusion and Rapid disease progression
#  Answer with only one of the options (Entailment or Contradiction):"""
 #Contradiction

# input_text="""Primary trial evidence are INTERVENTION 1:   Gefitinib (ZD1839)  ZD1839 at a daily dose of 250 mg or 500 mg depending on final dose in parent trial Secondary trial evidence are INTERVENTION 1:   Zoledronic Acid  Zoledronic acid, vitamin D and calcium supplements.INTERVENTION 2:   Zoledronic Acid + Radiopharmaceuticals  Zoledronic acid, vitamin D and calcium supplements, plus Sr-89 or Sm-153.
# Question: Does this imply that
# The intervention in the primary trial consists of a single drug, whereas in the secondary trial the intervention requires at least 3 different drugs?
# Answer with only one of the options (Entailment or Contradiction):"""
#Entailment

# input_text = """
# Primary trial evidence are INTERVENTION 1:   Laser Therapy Alone therapist administered laser treatment laser: therapist administered laserINTERVENTION 2: Mld Alone therapist administered manual lymphatic drainage manual lymphatic drainage: therapist administered massage therapy Secondary trial evidence are INTERVENTION 1: Part A Abemaciclib: HR+, HER2+ Breast Cancer Abemaciclib 200 mg was administered orally once every 12 hours on days 1-21 of a 21-day cycle when administered as a single agent or in combination with endocrine therapy (ET). Participants with hormone receptor positive HR+, HER2+ breast cancer receiving concurrent trastuzumab, 150 mg abemaciclib was given orally once every 12 hours on days 1-21 of a 21-day cycle. Participants may continue to receive treatment until discontinuation criteria are met.INTERVENTION 2: Part B Abemaciclib: HR+, HER2- Breast Cancer Abemaciclib 200 mg was administered orally once every 12 hours on days 1-21 of a 21-day cycle when administered as a single agent or for in combination with endocrine therapy (ET). Participants may continue to receive treatment until discontinuation criteria are met.
# Question: Does this imply that Laser Therapy is in each cohort of the primary trial and the secondary trial, along with neoadjuvant chemotherapy?
# Answer with only one of the options (Entailment or Contradiction):
# """
#Contradiction

# input_text= """Primary trial evidence are: ['Inclusion Criteria:', '  Voluntarily signed and dated written informed consent', '  Age between 18 and 75 years old (both inclusive)', '  Eastern Cooperative Oncology Group (ECOG) performance status (PS) of  1', '  Life expectancy  3 months.', '  Patients with a histologically/cytologically confirmed diagnosis of advanced and/or unresectable disease of any of the following tumors:', '  Breast cancer', '  Epithelial ovarian cancer or gynecological cancer', '  Head and neck squamous cell carcinoma', '  Non-small cell lung cancer', '  Small cell lung cancer', '  Platinum-refractory germ-cell tumors.', '  Adenocarcinoma or carcinoma of unknown primary site', '  Adequate bone marrow, renal, hepatic, and metabolic function', '  Recovery to grade  1 or to baseline from any Adverse Event (AE) derived from previous treatment (excluding alopecia of any grade).', '  Pre-menopausal women must have a negative pregnancy test before study entry and agree to use a medically acceptable method of contraception throughout the treatment period and for at least six weeks after treatment discontinuation', 'Exclusion Criteria:', '  Prior treatment with PM01183 or weekly paclitaxel or nanoalbumin-paclitaxel', '  Patients who have previously discontinued paclitaxel-based regimes due to drug related toxicity.', '  Known hypersensitivity to bevacizumab or any component of its formulation', '  Patients who have previously discontinued bevacizumab-containing regimes due to drug-related toxicity.', '  More than three prior lines of chemotherapy', '  Less than three months since last taxane-containing therapy.', '  Wash-out period:', '  Less than three weeks since the last chemotherapy-containing regimen', '  Less than three weeks since the last radiotherapy dose', '  Less than four weeks since last monoclonal antibody-containing therapy', '  Concomitant diseases/conditions:', '  Unstable angina, myocardial infarction, valvular heart disease, encephalopathy, ischemic attacks, hemorrhagic or ischemic cerebrovascular accident (CVA) or ongoing pulmonary embolism within last year, arrhythmia, hepatopathy, uncontrolled infection, hemoptysis or oxygen requiring dyspnea, known HIV infection, bleeding risk, muscular problems, peripheral neuropathy, Symptomatic or progressive brain metastases or leptomeningeal disease.', '  Men or pre-menopausal women who are not using an effective method of contraception as previously described; actively breast feeding women.', '  Patients who have pelvic irradiation with doses  45 Grays (Gy).', '  History of previous bone marrow and/or stem cell transplantation.', '  Confirmed bone marrow involvement']
# Question: does this imply that Patients with a histologically/cytologically confirmed diagnosis of a resectable Non-small cell lung breast cancer or Small cell lung cancer are eligible for the primary trial
# Answer with only one of the options (Entailment or Contradiction):"""
# contradiction



# input_text= """Primary trial evidence are Adverse Events 1:  Total: 5/32 (15.63%)  Febrile neutropenia 1/32 (3.13%)  Supraventricular tachycardia 1/32 (3.13%)  Hypersensitivity 2/32 (6.25%)  Catheter site infection 1/32 (3.13%)  Confusional state 1/32 (3.13%) Secondary trial evidence are Adverse Events 1:  Total: 285/752 (37.90%)  Anaemia 2/752 (0.27%)  Disseminated intravascular coagulation 2/752 (0.27%)  Febrile neutropenia 51/752 (6.78%)  Neutropenia 47/752 (6.25%)  Thrombocytopenia 2/752 (0.27%)  Atrial fibrillation 1/752 (0.13%)  Atrial flutter 0/752 (0.00%)  Cardiac failure congestive 1/752 (0.13%)  Left ventricular dysfunction 0/752 (0.00%)Adverse Events 2:  Total: 117/382 (30.63%)  Anaemia 3/382 (0.79%)  Disseminated intravascular coagulation 0/382 (0.00%)  Febrile neutropenia 11/382 (2.88%)  Neutropenia 20/382 (5.24%)  Thrombocytopenia 0/382 (0.00%)  Atrial fibrillation 1/382 (0.26%)  Atrial flutter 1/382 (0.26%)  Cardiac failure congestive 0/382 (0.00%)  Left ventricular dysfunction 1/382 (0.26%) "
# Question: does this imply that Heart-related adverse events were recorded in both the primary trial and the secondary trial.
# Answer with only one of the options (Entailment or Contradiction):"""
# # Entailment



# input_text= """Primary trial evidence are Inclusion Criteria: Age 52-75 years old; Identification as Latina/Hispanic/Chicana female; Residence in Pilsen, Little Village, East Side or South Chicago; No history of health volunteerism; No history of breast cancer; and Lack of a mammogram within the last two yearsExclusion Criteria: Not meeting all inclusion criteria; Women will be excluded if they participated in formative focus groups."
# Question: does this imply that
# Patients eligible for the primary trial must live in the USA.
# Answer with only one of the options (Entailment or Contradiction):"""
#Entailment

# input_text= """Primary trial evidence are Adverse Events 1: Total: 10/71 (14.08%) ATRIAL FIBRILLATION 1/71 (1.41%) CARDIAC TAMPONADE 1/71 (1.41%) PERICARDIAL EFFUSION 1/71 (1.41%) SUPRAVENTRICULAR TACHYCARDIA 1/71 (1.41%) DIARRHOEA 1/71 (1.41%) NAUSEA 1/71 (1.41%) VOMITING 1/71 (1.41%) CHEST PAIN 1/71 (1.41%) PNEUMONIA 1/71 (1.41%) MALIGNANT PLEURAL EFFUSION 1/71 (1.41%) HEPATIC ENCEPHALOPATHY 1/71 (1.41%) .
# Question: does this imply that
# There were only 3 adverse events in the primary trial which occurred more than twice?
# Answer with only one of the options (Entailment or Contradiction):"""
# "Contradiction"

# 3. FLAN-T5_2 shots

In [ ]:
def get_input_text_2shot(premise, hypothesis):
    options_prefix = "OPTIONS:\n- "
    separator = "\n- "
    options_ = options_prefix + f"{separator}".join(["Entailment", "Contradiction"])

    # Two-shot examples
    example_1_premise="Primary trial evidence are Inclusion Criteria: Age 52-75 years old; Identification as Latina/Hispanic/Chicana female; Residence in Pilsen, Little Village, East Side or South Chicago; No history of health volunteerism; No history of breast cancer; and Lack of a mammogram within the last two yearsExclusion Criteria: Not meeting all inclusion criteria; Women will be excluded if they participated in formative focus groups Question: uestion: Does this imply that {hypothesis}? Please answer by reasoning step by step and providing a final answer. OPTIONS: - Entailment - Contradiction"
    example_1_hypothesis="Patients eligible for the primary trial must live in the USA."
    example_1_answer="Entailment"

    example_2_premise="Primary trial evidence are Adverse Events 1: Total: 10/71 (14.08%) ATRIAL FIBRILLATION 1/71 (1.41%) CARDIAC TAMPONADE 1/71 (1.41%) PERICARDIAL EFFUSION 1/71 (1.41%) SUPRAVENTRICULAR TACHYCARDIA 1/71 (1.41%) DIARRHOEA 1/71 (1.41%) NAUSEA 1/71 (1.41%) VOMITING 1/71 (1.41%) CHEST PAIN 1/71 (1.41%) PNEUMONIA 1/71 (1.41%) MALIGNANT PLEURAL EFFUSION 1/71 (1.41%) HEPATIC ENCEPHALOPATHY 1/71 (1.41%) Question: Does this imply that {hypothesis}? Please answer by reasoning step by step and providing a final answer. OPTIONS: - Entailment - Contradiction"
    example_2_hypothesis="There were only 3 adverse events in the primary trial which occurred more than twice."
    example_2_answer="Contradiction"


   # Introduction text with examples
    example_intro = "Here are my 2 examples:\n"

    # Example text (Two-shot)
    two_shot_text = (
        f"{example_intro}"
        f"Example 1:\nPremise: {example_1_premise}\n"
        f"Hypothesis: {example_1_hypothesis}\n"
        f"Answer: {example_1_answer}\n\n"
        f"Example 2:\nPremise: {example_2_premise}\n"
        f"Hypothesis: {example_2_hypothesis}\n"
        f"Answer: {example_2_answer}\n\n")


    # For the actual input
    input_text = f"{premise} \nQuestion: Does this imply that {hypothesis}? Just provide the final answer.\n{options_}"

    # Combine two-shot examples with the actual input
    return two_shot_text + input_text
# Call the process_samples function
samples_2shot = process_samples("./test_data_new_format.xlsx", get_input_text_2shot)

# Output the samples list
# print(samples)


In [ ]:
labels = []
pred = []
with torch.inference_mode():
    for samples_2shot in tqdm.tqdm(samples_2shot):
        labels.append(samples_2shot["label"])
        input_ids = tokenizer(samples_2shot["text"], return_tensors="pt")
        # print(input_ids)

        outputs = model.generate(**input_ids)
        # pred.append(tokenizer.decode(outputs[0]))
        pred.append(tokenizer.batch_decode(outputs))

In [ ]:

# Load the Excel file into a DataFrame
d_path="methods_output_test_data.xlsx"
df = pd.read_excel(d_path)

# Add the new column to the DataFrame
df['two_shot_base_s_ex'] = pred
df['two_shot_base_clean_pred_s_ex'] = clean_pred(pred)

# Save the updated DataFrame back to an Excel file
df.to_excel(d_path, index=False)

df

In [ ]:
save_path = './data/two_shot_test_data_s_ex.json'
with open(save_path, 'w') as f:
    json.dump(samples_2shot, f)

In [ ]:
import evaluate
result_table = evaluate.evaluate_all_methods('./methods_output_test_data.xlsx','./results_test_data.xlsx')
result_table

# 4.FLAN-T5_cot

In [ ]:
def get_input_text_cot(premise, hypothesis):
    options_prefix = "OPTIONS:\n- "
    separator = "\n- "
    options_ = options_prefix + f"{separator}".join(["Entailment", "Contradiction"])

    # For the actual input
    input_text = f"{premise} \nQuestion: Does this imply that {hypothesis}? Please answer by reasoning step by step and providing a final answer.\n{options_}"

    # Combine two-shot examples with the actual input
    return input_text

In [ ]:
samples_cot = []
item=[]
for item in data_expanded:
    primary_evidence = "".join(item['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = item.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(item['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text =get_input_text_cot(sentence, item['statement'])
    temp = {"text":input_text, "label":item['label']}
    samples_cot.append(temp)

In [ ]:
samples_cot

In [ ]:
labels = []
pred = []
samples= samples_cot
with torch.inference_mode():
    for samples in tqdm.tqdm(samples):
        labels.append(samples["label"])
        input_ids = tokenizer(samples["text"], return_tensors="pt")
        # print(input_ids)

        outputs = model.generate(**input_ids, max_length = 1024)
        # pred.append(tokenizer.decode(outputs[0]))
        pred.append(tokenizer.batch_decode(outputs))

In [ ]:
pred

In [ ]:
cleaned_pred = []
data=pred
for sentence in data:
    # Extract the string from the list (assuming each sentence is a list with one element)
    if isinstance(sentence, list):
        sentence = sentence[0]

    # Strip unwanted tokens and spaces
    sentence = sentence.replace("<pad>", "").replace("</s>", "").strip()
    # Now check for "Contradiction" or "Entailment"
    if "Contradiction" in sentence:
        cleaned_pred.append("Contradiction")
    elif "Entailment" in sentence:
        cleaned_pred.append("Entailment")

In [ ]:
cleaned_pred

In [ ]:
# Compute metrics with different averaging methods
metrics_micro = compute_metrics(labels, cleaned_pred, averaging="micro")
metrics_macro = compute_metrics(labels, cleaned_pred, averaging="macro")
metrics_weighted = compute_metrics(labels, cleaned_pred, averaging="weighted")
#save_metrics_to_csv('FLAN-T5-COT', metrics_micro, metrics_macro, metrics_weighted, result_path)
# Print results
print("Micro averaging:", metrics_micro)
print("Macro averaging:", metrics_macro)
print("Weighted averaging:", metrics_weighted)

In [ ]:
# Compute metrics with different averaging methods
pred=cleaned_predictions
metrics_micro = compute_metrics(labels, pred, averaging="micro")
metrics_macro = compute_metrics(labels, pred, averaging="macro")
metrics_weighted = compute_metrics(labels, pred, averaging="weighted")

# Print results
print("Micro averaging:", metrics_micro)
print("Macro averaging:", metrics_macro)
print("Weighted averaging:", metrics_weighted)

# Micro averaging: {'precision': 0.545, 'recall': 0.545, 'f1_score': 0.545}
# Macro averaging: {'precision': 0.5608190295985944, 'recall': 0.545, 'f1_score': 0.5133559720848151}
# Weighted averaging: {'precision': 0.5608190295985944, 'recall': 0.545, 'f1_score': 0.5133559720848151}

# cot
# Micro averaging: {'precision': 0.52, 'recall': 0.52, 'f1_score': 0.52}
# Macro averaging: {'precision': 0.5229779411764706, 'recall': 0.52, 'f1_score': 0.5039272426622571}
# Weighted averaging: {'precision': 0.5229779411764706, 'recall': 0.52, 'f1_score': 0.5039272426622571}

#return f"{premise} \nQuestion: Does this imply that {hypothesis}? Please answer by reasoning step by step and providing a final answer.\n{options_}"
# Micro averaging: {'precision': 0.53, 'recall': 0.53, 'f1_score': 0.53}
# Macro averaging: {'precision': 0.18177387914230017, 'recall': 0.17666666666666667, 'f1_score': 0.1778885045007831} ???
# Weighted averaging: {'precision': 0.5453216374269005, 'recall': 0.53, 'f1_score': 0.5336655135023493}

#########################
# #tf_idf:
# F1:0.502415
# precision_score:0.485981
# recall_score:0.520000

########   Question: Does this imply that {hypothesis}?
# #direct
# return f"{premise} \nQuestion: Does this imply that {hypothesis}? Just provide the final answer.\n{options_}"
# Micro averaging: {'precision': 0.52, 'recall': 0.52, 'f1_score': 0.52}
# Macro averaging: {'precision': 0.53125, 'recall': 0.52, 'f1_score': 0.4725274725274725}
# Weighted averaging: {'precision': 0.53125, 'recall': 0.52, 'f1_score': 0.47252747252747246}

# #COT
##return f"{premise} \nQuestion: Does this imply that {hypothesis}? Please answer by reasoning step by step and providing a final answer.\n{options_}"
#after adding the posstprossing function:
# Micro averaging: {'precision': 0.54, 'recall': 0.54, 'f1_score': 0.54}
# Macro averaging: {'precision': 0.5407996736026112, 'recall': 0.54, 'f1_score': 0.5377349010149733}
# Weighted averaging: {'precision': 0.5407996736026112, 'recall': 0.54, 'f1_score': 0.5377349010149733}

#cot without options:
# return f"{premise} \nQuestion: Does this imply that {hypothesis}? Please answer by reasoning step by step and providing a final answer."
# Micro averaging: {'precision': 0.495, 'recall': 0.495, 'f1_score': 0.495}
# Macro averaging: {'precision': 0.44871794871794873, 'recall': 0.495, 'f1_score': 0.3478611783696529}
# Weighted averaging: {'precision': 0.44871794871794873, 'recall': 0.495, 'f1_score': 0.34786117836965297}



######    Read the following text and determine if the sentence is true ...
#direct
# If you prefer a {options_}
# return f"Read the following text and determine if the sentence is true.\nText: {premise} \nSentence: {hypothesis} \nProvide the final answer: {options_}."
# Micro averaging: {'precision': 0.515, 'recall': 0.515, 'f1_score': 0.515}
# Macro averaging: {'precision': 0.5197394393999211, 'recall': 0.515, 'f1_score': 0.48402883055400414}
# Weighted averaging: {'precision': 0.5197394393999211, 'recall': 0.515, 'f1_score': 0.4840288305540041}

#cot:
# return f"Read the following text and determine if the sentence is true.\nText: {premise} \nSentence: {hypothesis} \nPlease answer by reasoning step by step and providing a final answer.\n{options_}"
# Micro averaging: {'precision': 0.52, 'recall': 0.52, 'f1_score': 0.52}
# Macro averaging: {'precision': 0.5233754090696587, 'recall': 0.52, 'f1_score': 0.5020230314347961}
# Weighted averaging: {'precision': 0.5233754090696587, 'recall': 0.52, 'f1_score': 0.5020230314347961}

# result: return f"{premise} \nQuestion: Does this imply that {hypothesis}? Please answer by reasoning step by step and providing a final answer.\n{options_}"
# Micro averaging: {'precision': 0.52, 'recall': 0.52, 'f1_score': 0.52}
# Macro averaging: {'precision': 0.5233754090696587, 'recall': 0.52, 'f1_score': 0.5020230314347961}
# Weighted averaging: {'precision': 0.5233754090696587, 'recall': 0.52, 'f1_score': 0.5020230314347961}


# #######two shots:
#  two shots:+ cot
#     return (
#     f"Here are two examples of input and answer:\n"
#     f"Example 1: Input: {text_1}, Answer: {label_1}\n"
#     f"Example 2: Input: {text_2}, Answer: {label_2}\n\n"
#     f"Now consider the following:\nPremise: {premise}\n"
#     f"Question: Does this imply that \"{hypothesis}\"?\n"
#     f"Please provide your reasoning step by step, and conclude with a final answer.\n{options_}"


# Micro averaging: {'precision': 0.52, 'recall': 0.52, 'f1_score': 0.52}
# Macro averaging: {'precision': 0.5233754090696587, 'recall': 0.52, 'f1_score': 0.5020230314347961}
# Weighted averaging: {'precision': 0.5233754090696587, 'recall': 0.52, 'f1_score': 0.5020230314347961}



# 5. FLAN-T5_cot+2shot

In [ ]:
def get_input_text_cot_2shot(premise, hypothesis):
    options_prefix = "OPTIONS:\n- "
    separator = "\n- "
    options_ = options_prefix + f"{separator}".join(["Entailment", "Contradiction"])

    # Two-shot examples
    example_1_premise = "Primary trial evidence are INTERVENTION 1:   Laser Therapy Alone  therapist administered laser treatment  laser: therapist administered laserINTERVENTION 2:   Mld Alone  therapist administered manual lymphatic drainage  manual lymphatic drainage: therapist administered massage therapy Secondary trial evidence are INTERVENTION 1:   Part A Abemaciclib: HR+, HER2+ Breast Cancer  Abemaciclib 200 mg was administered orally once every 12 hours on days 1-21 of a 21-day cycle when administered as a single agent or in combination with endocrine therapy (ET). Participants with hormone receptor positive HR+, HER2+ breast cancer receiving concurrent trastuzumab, 150 mg abemaciclib was given orally once every 12 hours on days 1-21 of a 21-day cycle. Participants may continue to receive treatment until discontinuation criteria are met.INTERVENTION 2:   Part B Abemaciclib: HR+, HER2- Breast Cancer  Abemaciclib 200 mg was administered orally once every 12 hours on days 1-21 of a 21-day cycle when administered as a single agent or for in combination with endocrine therapy (ET).  Participants may continue to receive treatment until discontinuation criteria are met."
    example_1_hypothesis = "Laser Therapy is in each cohort of the primary trial and the secondary trial, along with neoadjuvant chemotherapy. "
    example_1_answer = "Contradiction"

    example_2_premise = "Primary trial evidence are Adverse Events 1:  Total: 5/32 (15.63%)  Febrile neutropenia 1/32 (3.13%)  Supraventricular tachycardia 1/32 (3.13%)  Hypersensitivity 2/32 (6.25%)  Catheter site infection 1/32 (3.13%)  Confusional state 1/32 (3.13%) Secondary trial evidence are Adverse Events 1:  Total: 285/752 (37.90%)  Anaemia 2/752 (0.27%)  Disseminated intravascular coagulation 2/752 (0.27%)  Febrile neutropenia 51/752 (6.78%)  Neutropenia 47/752 (6.25%)  Thrombocytopenia 2/752 (0.27%)  Atrial fibrillation 1/752 (0.13%)  Atrial flutter 0/752 (0.00%)  Cardiac failure congestive 1/752 (0.13%)  Left ventricular dysfunction 0/752 (0.00%)Adverse Events 2:  Total: 117/382 (30.63%)  Anaemia 3/382 (0.79%)  Disseminated intravascular coagulation 0/382 (0.00%)  Febrile neutropenia 11/382 (2.88%)  Neutropenia 20/382 (5.24%)  Thrombocytopenia 0/382 (0.00%)  Atrial fibrillation 1/382 (0.26%)  Atrial flutter 1/382 (0.26%)  Cardiac failure congestive 0/382 (0.00%)  Left ventricular dysfunction 1/382 (0.26%) "
    example_2_hypothesis = "Heart-related adverse events were recorded in both the primary trial and the secondary trial."
    example_2_answer = "Entailment"

   # Introduction text with examples
    example_intro = "Here are my 2 examples:\n"

    # Example text (Two-shot)
    two_shot_text = (
        f"{example_intro}"
        f"Example 1:\nPremise: {example_1_premise}\n"
        f"Hypothesis: {example_1_hypothesis}\n"
        f"Answer: {example_1_answer}\n\n"
        f"Example 2:\nPremise: {example_2_premise}\n"
        f"Hypothesis: {example_2_hypothesis}\n"
        f"Answer: {example_2_answer}\n\n")


    # For the actual input
    input_text = f"{premise} \nQuestion: Does this imply that {hypothesis}? Please answer by reasoning step by step and providing a final answer.\n{options_}"

    # Combine two-shot examples with the actual input
    return two_shot_text + input_text

In [ ]:
samples_cot = []
item=[]
for item in data_expanded:
    primary_evidence = "".join(item['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = item.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(item['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text =get_input_text_cot_2shot(sentence, item['statement'])
    temp = {"text":input_text, "label":item['label']}
    samples_cot.append(temp)

In [ ]:
labels = []
pred = []
samples= samples_cot
with torch.inference_mode():
    for samples in tqdm.tqdm(samples):
        labels.append(samples["label"])
        input_ids = tokenizer(samples["text"], return_tensors="pt")
        # print(input_ids)

        outputs = model.generate(**input_ids, max_length = 1024)
        # pred.append(tokenizer.decode(outputs[0]))
        pred.append(tokenizer.batch_decode(outputs))

In [ ]:
pred

In [ ]:
cleaned_pred = []
data=pred
for sentence in data:
    # Extract the string from the list (assuming each sentence is a list with one element)
    if isinstance(sentence, list):
        sentence = sentence[0]

    # Strip unwanted tokens and spaces
    sentence = sentence.replace("<pad>", "").replace("</s>", "").strip()
    # Now check for "Contradiction" or "Entailment"
    if "Contradiction" in sentence:
        cleaned_pred.append("Contradiction")
    elif "Entailment" in sentence:
        cleaned_pred.append("Entailment")

In [ ]:
cleaned_pred

In [ ]:
metrics_micro = compute_metrics(labels, cleaned_pred, averaging="micro")
metrics_macro = compute_metrics(labels, cleaned_pred, averaging="macro")
metrics_weighted = compute_metrics(labels, cleaned_pred, averaging="weighted")
save_metrics_to_csv('FLAN-T5-COT+2shot', metrics_micro, metrics_macro, metrics_weighted, result_path)
# Print results
print("Micro averaging:", metrics_micro)
print("Macro averaging:", metrics_macro)
print("Weighted averaging:", metrics_weighted)

# 6. FLAN-T5 FINE-TUNE

In [ ]:
file_path='/content/drive/MyDrive/sepi_master_thesis/data/'

train_path = file_path + 'combined_train2.json'
with open(train_path) as json_file:
    train_data = json.load(json_file)# training data (list of dicts with "text" and "label")

dev_path = file_path + 'combined_dev2.json'
with open(dev_path) as json_file:
    dev_data = json.load(json_file)  #  dev/validation data

test_path = file_path + 'combined_test2.json'
with open(test_path) as json_file:
    test_data = json.load(json_file)  #  test data



In [ ]:
test_data[50]

In [ ]:
import torch
import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from google.colab import drive


# Load model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
epochs = 4  # Number of epochs
# train_data
# dev_data
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Training phase
    model.train()
    for sample in tqdm.tqdm(train_data, desc="Training"):
        input_ids = tokenizer(sample["text"], return_tensors="pt", truncation=True, padding=True, max_length=512).input_ids.to(device)
        labels = tokenizer(sample["label"], return_tensors="pt", truncation=True, padding=True, max_length=512).input_ids.to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluation phase (optional)
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for sample in dev_data:
            input_ids = tokenizer(sample["text"], return_tensors="pt", truncation=True, padding=True, max_length=512).input_ids.to(device)
            labels = tokenizer(sample["label"], return_tensors="pt", truncation=True, padding=True, max_length=512).input_ids.to(device)

            # Forward pass (no backprop)
            outputs = model(input_ids=input_ids, labels=labels)
            total_loss += outputs.loss.item()

    print(f"Epoch {epoch + 1} - Evaluation Loss: {total_loss / len(dev_data):.4f}")

# Save the fine-tuned model to Google Drive
save_path = '/content/drive/MyDrive/sepi_master_thesis/large-x2/'

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


In [ ]:
model_path = "/content/drive/MyDrive/sepi_master_thesis/result_25.11.2024/models/large"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
labels = []
pred = []
samples= test_data
with torch.inference_mode():
    for samples in tqdm.tqdm(samples):
        labels.append(samples["label"])
        input_ids = tokenizer(samples["text"], return_tensors="pt")
        # print(input_ids)

        outputs = model.generate(**input_ids, max_length = 1024)
        # pred.append(tokenizer.decode(outputs[0]))
        pred.append(tokenizer.batch_decode(outputs))

In [ ]:
pred

In [ ]:

# Load the Excel file
file_path = '/content/drive/MyDrive/sepi_master_thesis/result_25.11.2024/methods_output_test_data2.xlsx' # Replace with your Excel file path
df = pd.read_excel(file_path)
# Define the value for the new column
new_column_value = cleaned_pred  # This will populate the new column

# Add the new column named "fine-tuned" to the DataFrame
df["fine-tuned_largex2_clean_pred"] = new_column_value

# Save the updated DataFrame back to an Excel file
output_path = '/content/drive/MyDrive/sepi_master_thesis/result_25.11.2024/methods_output_test_data2.xlsx'  # Path for the updated Excel file
df.to_excel(output_path, index=False)

print(f"New column added and saved to {output_path}")

In [ ]:
cleaned_pred = []
data=pred
for sentence in data:
    # Extract the string from the list (assuming each sentence is a list with one element)
    if isinstance(sentence, list):
        sentence = sentence[0]

    # Strip unwanted tokens and spaces
    sentence = sentence.replace("<pad>", "").replace("</s>", "").strip()
    # Now check for "Contradiction" or "Entailment"
    if "Contradiction" in sentence:
        cleaned_pred.append("Contradiction")
    elif "Entailment" in sentence:
        cleaned_pred.append("Entailment")


In [ ]:
cleaned_pred

In [ ]:
# Compute metrics with different averaging methods
metrics_micro = compute_metrics(labels, cleaned_pred, averaging="micro")
metrics_macro = compute_metrics(labels, cleaned_pred, averaging="macro")
metrics_weighted = compute_metrics(labels, cleaned_pred, averaging="weighted")
save_metrics_to_csv('FLAN-T5_fine-tune', metrics_micro, metrics_macro, metrics_weighted, result_path)
# Print results
print("Micro averaging:", metrics_micro)
print("Macro averaging:", metrics_macro)
print("Weighted averaging:", metrics_weighted)

# 3.  2_shot+ with COT:

In [ ]:
samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text(sentence, sample['statement'])
    temp = {"text":input_text, "label":sample['label']}
    samples.append(temp)

In [ ]:
#runing model:
labels = []
pred = []
# samples= samples[0:8]
with torch.inference_mode():
    for samples in tqdm.tqdm(samples):
        labels.append(samples["label"])
        input_ids = tokenizer(samples["text"], return_tensors="pt")
        # print(input_ids)

        outputs = model.generate(**input_ids, max_length = 1024)
        # pred.append(tokenizer.decode(outputs[0]))
        pred.append(tokenizer.batch_decode(outputs))

In [ ]:
# Compute metrics with different averaging methods
pred=cleaned_predictions
metrics_micro = compute_metrics(labels, pred, averaging="micro")
metrics_macro = compute_metrics(labels, pred, averaging="macro")
metrics_weighted = compute_metrics(labels, pred, averaging="weighted")

# Print results
print("Micro averaging:", metrics_micro)
print("Macro averaging:", metrics_macro)
print("Weighted averaging:", metrics_weighted)


#########################
# #tf_idf:
# F1:0.502415
# precision_score:0.485981
# recall_score:0.520000

########   Question: Does this imply that {hypothesis}?
# #direct
# return f"{premise} \nQuestion: Does this imply that {hypothesis}? Just provide the final answer.\n{options_}"
# Micro averaging: {'precision': 0.52, 'recall': 0.52, 'f1_score': 0.52}
# Macro averaging: {'precision': 0.53125, 'recall': 0.52, 'f1_score': 0.4725274725274725}
# Weighted averaging: {'precision': 0.53125, 'recall': 0.52, 'f1_score': 0.47252747252747246}

##COT
##return f"{premise} \nQuestion: Does this imply that {hypothesis}? Please answer by reasoning step by step and providing a final answer.\n{options_}"
#after adding the posstprossing function:
# Micro averaging: {'precision': 0.54, 'recall': 0.54, 'f1_score': 0.54}
# Macro averaging: {'precision': 0.5407996736026112, 'recall': 0.54, 'f1_score': 0.5377349010149733}
# Weighted averaging: {'precision': 0.5407996736026112, 'recall': 0.54, 'f1_score': 0.5377349010149733}



# #######two shots:
#  two shots:+ cot
#     return (
#     f"Here are two examples of input and answer:\n"
#     f"Example 1: Input: {text_1}, Answer: {label_1}\n"
#     f"Example 2: Input: {text_2}, Answer: {label_2}\n\n"
#     f"Now consider the following:\nPremise: {premise}\n"
#     f"Question: Does this imply that \"{hypothesis}\"?\n"
#     f"Please provide your reasoning step by step, and conclude with a final answer.\n{options_}"


# Micro averaging: {'precision': 0.52, 'recall': 0.52, 'f1_score': 0.52}
# Macro averaging: {'precision': 0.5233754090696587, 'recall': 0.52, 'f1_score': 0.5020230314347961}
# Weighted averaging: {'precision': 0.5233754090696587, 'recall': 0.52, 'f1_score': 0.5020230314347961}


# fine tunning

# Micro averaging: {'precision': 0.548, 'recall': 0.548, 'f1_score': 0.548}
# Macro averaging: {'precision': 0.549016404156591, 'recall': 0.548, 'f1_score': 0.5456446217189912}
# Weighted averaging: {'precision': 0.549016404156591, 'recall': 0.548, 'f1_score': 0.5456446217189913}

# sepi's results
## TFIDF
F1:0.492424

precision_score:0.467626

recall_score:0.520000

## ZERO SHOT

Question :{premise} \n Question: Does this imply that {hypothesis}? {options_}

Micro averaging: {'precision': 0.49 ,    'recall':0.49,     'f1_score': 0.49}

Macro averaging: {'precision': 0.3247814735,    'recall': 0.3266666667,
   'f1_score': 0.3005619396}

Weighted averaging: {'precision': 0.4871722102,    'recall': 0.49,
   'f1_score': 0.4508429094}

## TWO SHO

Question: "Here are my 2 examples:"Example 1:Premise-Hypothesis-Answer
           Example 2:Premise-Hypothesis-Answer/{premise} \nQuestion: Does this imply that {hypothesis}? Just provide the final answer.\n{options_}  "
  

Micro averaging: {'precision': 0.518, 'recall': 0.518, 'f1_score': 0.518}

Macro averaging: {'precision': 0.520612323421096, 'recall': 0.518, 'f1_score': 0.5022286113210976}

Weighted averaging: {'precision': 0.5206123234210961, 'recall': 0.518, 'f1_score': 0.5022286113210976}


## COT

Question:

Micro averaging: {'precision': 0.512, 'recall': 0.512, 'f1_score': 0.512}

Macro averaging: {'precision': 0.5120069159836065, 'recall': 0.512, 'f1_score': 0.5119297178793747}

Weighted averaging: {'precision': 0.5120069159836065, 'recall': 0.512, 'f1_score': 0.5119297178793746}

##cot+2shot
Micro averaging: {'precision': 0.504, 'recall': 0.504, 'f1_score': 0.504}

Macro averaging: {'precision': 0.5066602728047741, 'recall': 0.504, 'f1_score': 0.44897705256047915}

Weighted averaging: {'precision': 0.5066602728047741, 'recall': 0.504, 'f1_score': 0.4489770525604792}



##finetuned

Micro averaging: {'precision': 0.548, 'recall': 0.548, 'f1_score': 0.548}

Macro averaging: {'precision': 0.549016404156591, 'recall': 0.548, 'f1_score': 0.5456446217189912}

Weighted averaging: {'precision': 0.549016404156591, 'recall': 0.548, 'f1_score': 0.5456446217189913}



# Mistral-7B


In [ ]:
!pip install transformers
from transformers import pipeline

In [ ]:
!huggingface-cli login

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
df

In [ ]:
data=df

In [ ]:
data

In [ ]:
from tqdm import tqdm

model = model.to('cuda')

responses = []

for input_text in tqdm(data['text'], desc="Processing inputs", total=len(data['text'])):

    inputs = tokenizer(input_text, return_tensors="pt").to('cuda')

    outputs = model.generate(
        **inputs,
        max_new_tokens=5,  # Generate a short response
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    responses.append(response)

data['model_response_mistral_twoshots'] = responses

In [ ]:
responses[499]

In [ ]:
import pandas as pd

responses_df = pd.DataFrame(responses, columns=["model_response"])

responses_df.to_csv('/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/responses-two-shot-mistral.csv', index=False)

In [ ]:
responses_df

In [ ]:
labels = []

#responses = df['mistral-7b-zero-shot']
#responses= q1['prediction']
for idx, response in enumerate(responses):  # Ensure idx is properly defined
    tokens = response.split()

    # Check if the last token is valid
    if tokens and tokens[-1] in ["Entailment", "Contradiction"]:
        labels.append(tokens[-1])  # Use the last token if valid
    else:
        labels.append('nan')

labels_df = pd.DataFrame(labels, columns=["label"])

In [ ]:
labels_df

In [ ]:
file_path = "//content/drive/MyDrive/pycharm_results/test_zero_shot_qwen.csv"
df= pd.read_csv(file_path)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics(true_labels, predicted_labels):
    """
    Calculate precision, recall, and F1 scores (weighted, micro, and macro).

    Args:
        true_labels (list or pd.Series): True labels.
        predicted_labels (list or pd.Series): Predicted labels.

    Returns:
        dict: A dictionary containing precision, recall, and F1 scores
              for weighted, micro, and macro averaging.
    """
    metrics = {}

    # Weighted metrics
    metrics['precision_weighted'] = precision_score(true_labels, predicted_labels, average='weighted')
    metrics['recall_weighted'] = recall_score(true_labels, predicted_labels, average='weighted')
    metrics['f1_weighted'] = f1_score(true_labels, predicted_labels, average='weighted')

    # Micro metrics
    metrics['precision_micro'] = precision_score(true_labels, predicted_labels, average='micro')
    metrics['recall_micro'] = recall_score(true_labels, predicted_labels, average='micro')
    metrics['f1_micro'] = f1_score(true_labels, predicted_labels, average='micro')

    # Macro metrics
    metrics['precision_macro'] = precision_score(true_labels, predicted_labels, average='macro')
    metrics['recall_macro'] = recall_score(true_labels, predicted_labels, average='macro')
    metrics['f1_macro'] = f1_score(true_labels, predicted_labels, average='macro')

    return metrics


In [ ]:
df

In [ ]:
df['predicted_output'] = df['predicted_output'].str.capitalize()

In [ ]:
nan_rows = df[df['mistral_twoshots_lable'].astype(str) == 'nan']
print(nan_rows)


In [ ]:
print(nan_rows.index.tolist())


In [ ]:
responses[499]

In [ ]:
nan_rows

In [ ]:
metrics = calculate_metrics(df['label'], df['predicted_output'])

In [ ]:
metrics

In [ ]:
metrics #qwen zero shot

In [ ]:
df['label'] = df['label'].astype(str)
df['mistral_twoshots_lable'] = df['mistral_twoshots_lable'].astype(str)

In [ ]:
metrics = calculate_metrics(df['label'], df['mistral_twoshots_lable'])

In [ ]:
metrics

In [ ]:
metrics

In [ ]:
import argparse
import pandas as pd
import ast
import re
import openpyxl
from openpyxl.styles import Color, PatternFill
import string
from sklearn.metrics import f1_score, precision_score, recall_score
!pip install xlsxwriter


def compute_metrics(label, pred, averaging="micro"):
    """
    Compute precision, recall, and F1 score using the specified averaging method.

    Parameters:
    - labels: True labels
    - pred: Predicted labels
    - averaging: Averaging method ("micro", "macro", "weighted")

    Returns:
    - A dictionary with precision, recall, and F1 score
    """

    # Calculate the precision, recall, and F1 score

    precision = precision_score(label, pred, average=averaging)
    recall = recall_score(label, pred, average=averaging)
    f1 = f1_score(label, pred, average=averaging)

    # Return the results as a dictionary
    return precision, recall,f1



def evaluate_all_methods(input_path,  output_path ):
    # Read the Excel file into a DataFrame
    df = pd.read_csv(input_path)
    label=df["label"]

    # excluded_columns = df.columns[4:]
    #excluded_columns = ["TF_IDF_prediction"]

    # Then append the other columns that end with "clean_pred"
    excluded_columns = [col for col in df.columns if col.endswith("clean_pred")]

    # Initialize an empty DataFrame to store the results
    # results_df = pd.DataFrame(columns=['Column', 'Precision', 'Recall', 'F1-score'])
    results_df = pd.DataFrame(columns=['Column'])
    print(excluded_columns)
    for col in excluded_columns:
        # Compute metrics with different averaging methods

        mi_precision, mi_recall,mi_f1 = compute_metrics(label, df[col], averaging="micro")
        ma_precision, ma_recall,ma_f1 = compute_metrics(label, df[col], averaging="macro")


        # Append the results to the DataFrame
        results_df = pd.concat([results_df, pd.DataFrame({
            'Column': [col],
            'micro_Precision': [mi_precision],
            'micro_Recall': [mi_recall],
            'micro_F1-score': [mi_f1],
            'macro_Precision': [ma_precision],
            'macro_Recall': [ma_recall],
            'macro_F1-score': [ma_f1],


        })], ignore_index=True)


    report_to_excel(results_df,output_path)

    print(f"results_partial_match is saved in {output_path}!..." )

    return results_df


def report_to_excel(df, excel_file):
    """
    Use conditional formatting on report and write to excel.
    """
    writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
    workbook = writer.book
    df.to_excel(writer, sheet_name='Confusion Matrix')
    for worksheet in workbook.worksheets():
        max_row, max_col = df.shape
        for col in range(2,8):
            worksheet.conditional_format(1, col, max_row, col ,
                                         {'type': '3_color_scale', 'min_value': 0, 'max_value': 1})
    # writer.save()
    writer.close()



# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('-dp', '--input_path', default='./methods_output_test_data.xlsx')
#     parser.add_argument('-op', '--output_path', default='./results_test_data.xlsx')
#     args = parser.parse_args()


    # Example usage

# df = evaluate_all_methods(f"{args.input_path}",f"{args.output_path}" )
# print(df)
methods_output_file="/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/total.csv"
results_file="/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/results_test_data2.xlsx"
rf = evaluate_all_methods(methods_output_file,results_file )

rf



In [ ]:
import pandas as pd

# File path
file_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/responses-zero-shot-qwen.csv"

# Load the CSV file into a DataFrame
q1= pd.read_csv(file_path)

# Rename the column
# Replace 'old_column_name' with the current name and 'new_column_name' with the desired name
# df.rename(columns={'model_response': 'mistral-7b-zero-shot'}, inplace=True)
#df
# # Save the updated DataFrame back to the same file
#s.to_csv(file_path, index=False)

# print("Column renamed and file saved successfully!")

#total.csv


In [ ]:
q1

In [ ]:
data['qwen2-7b-zero-shot']=q1

In [ ]:
data

# Llama

In [ ]:
  #Primary trial evidence are INTERVENTION 1:   Gefitinib (ZD1839)  ZD1839 at a daily dose of 250 mg or 500 mg depending on final dose in parent trial Secondary trial evidence are INTERVENTION 1:   Zoledronic Acid  Zoledronic acid, vitamin D and calcium supplements.INTERVENTION 2:   Zoledronic Acid + Radiopharmaceuticals  Zoledronic acid, vitamin D and calcium supplements, plus Sr-89 or Sm-153.
   #\n Question: Does this imply that
   #The intervention in the primary trial consists of a single drug, whereas in the secondary trial the intervention requires at least 3 different drugs.
   #Entailment

    # example_1_premise = "Primary trial evidence are INTERVENTION 1:   Laser Therapy Alone  therapist administered laser treatment  laser: therapist administered laserINTERVENTION 2:   Mld Alone  therapist administered manual lymphatic drainage  manual lymphatic drainage: therapist administered massage therapy Secondary trial evidence are INTERVENTION 1:   Part A Abemaciclib: HR+, HER2+ Breast Cancer  Abemaciclib 200 mg was administered orally once every 12 hours on days 1-21 of a 21-day cycle when administered as a single agent or in combination with endocrine therapy (ET). Participants with hormone receptor positive HR+, HER2+ breast cancer receiving concurrent trastuzumab, 150 mg abemaciclib was given orally once every 12 hours on days 1-21 of a 21-day cycle. Participants may continue to receive treatment until discontinuation criteria are met.INTERVENTION 2:   Part B Abemaciclib: HR+, HER2- Breast Cancer  Abemaciclib 200 mg was administered orally once every 12 hours on days 1-21 of a 21-day cycle when administered as a single agent or for in combination with endocrine therapy (ET).  Participants may continue to receive treatment until discontinuation criteria are met."
    # example_1_hypothesis = "Laser Therapy is in each cohort of the primary trial and the secondary trial, along with neoadjuvant chemotherapy. "
    # example_1_answer = "Contradiction"

    # example_2_premise = "Primary trial evidence are Adverse Events 1:  Total: 5/32 (15.63%)  Febrile neutropenia 1/32 (3.13%)  Supraventricular tachycardia 1/32 (3.13%)  Hypersensitivity 2/32 (6.25%)  Catheter site infection 1/32 (3.13%)  Confusional state 1/32 (3.13%) Secondary trial evidence are Adverse Events 1:  Total: 285/752 (37.90%)  Anaemia 2/752 (0.27%)  Disseminated intravascular coagulation 2/752 (0.27%)  Febrile neutropenia 51/752 (6.78%)  Neutropenia 47/752 (6.25%)  Thrombocytopenia 2/752 (0.27%)  Atrial fibrillation 1/752 (0.13%)  Atrial flutter 0/752 (0.00%)  Cardiac failure congestive 1/752 (0.13%)  Left ventricular dysfunction 0/752 (0.00%)Adverse Events 2:  Total: 117/382 (30.63%)  Anaemia 3/382 (0.79%)  Disseminated intravascular coagulation 0/382 (0.00%)  Febrile neutropenia 11/382 (2.88%)  Neutropenia 20/382 (5.24%)  Thrombocytopenia 0/382 (0.00%)  Atrial fibrillation 1/382 (0.26%)  Atrial flutter 1/382 (0.26%)  Cardiac failure congestive 0/382 (0.00%)  Left ventricular dysfunction 1/382 (0.26%) "
    # example_2_hypothesis = "Heart-related adverse events were recorded in both the primary trial and the secondary trial."
    # example_2_answer = "Entailment"



    # example_3_premise="Primary trial evidence are Inclusion Criteria: Age 52-75 years old; Identification as Latina/Hispanic/Chicana female; Residence in Pilsen, Little Village, East Side or South Chicago; No history of health volunteerism; No history of breast cancer; and Lack of a mammogram within the last two yearsExclusion Criteria: Not meeting all inclusion criteria; Women will be excluded if they participated in formative focus groups."
    # example_3_hypothesis="Patients eligible for the primary trial must live in the USA."
    # example_3_answer="Entailment"

    # example_4_premise="Primary trial evidence are Adverse Events 1: Total: 10/71 (14.08%) ATRIAL FIBRILLATION 1/71 (1.41%) CARDIAC TAMPONADE 1/71 (1.41%) PERICARDIAL EFFUSION 1/71 (1.41%) SUPRAVENTRICULAR TACHYCARDIA 1/71 (1.41%) DIARRHOEA 1/71 (1.41%) NAUSEA 1/71 (1.41%) VOMITING 1/71 (1.41%) CHEST PAIN 1/71 (1.41%) PNEUMONIA 1/71 (1.41%) MALIGNANT PLEURAL EFFUSION 1/71 (1.41%) HEPATIC ENCEPHALOPATHY 1/71 (1.41%) ."
    # example_4_hypothesis="There were only 3 adverse events in the primary trial which occurred more than twice."
    # example_4_answer="Contradiction"

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Initialize tokenizer and model for LLaMA 2
model_name = "meta-llama/Llama-2-7b-hf"  # This is the LLaMA 2 7B model

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)




In [ ]:
import torch
# input_text= """Primary trial evidence are: ['Inclusion Criteria:', '  Voluntarily signed and dated written informed consent', '  Age between 18 and 75 years old (both inclusive)', '  Eastern Cooperative Oncology Group (ECOG) performance status (PS) of  1', '  Life expectancy  3 months.', '  Patients with a histologically/cytologically confirmed diagnosis of advanced and/or unresectable disease of any of the following tumors:', '  Breast cancer', '  Epithelial ovarian cancer or gynecological cancer', '  Head and neck squamous cell carcinoma', '  Non-small cell lung cancer', '  Small cell lung cancer', '  Platinum-refractory germ-cell tumors.', '  Adenocarcinoma or carcinoma of unknown primary site', '  Adequate bone marrow, renal, hepatic, and metabolic function', '  Recovery to grade  1 or to baseline from any Adverse Event (AE) derived from previous treatment (excluding alopecia of any grade).', '  Pre-menopausal women must have a negative pregnancy test before study entry and agree to use a medically acceptable method of contraception throughout the treatment period and for at least six weeks after treatment discontinuation', 'Exclusion Criteria:', '  Prior treatment with PM01183 or weekly paclitaxel or nanoalbumin-paclitaxel', '  Patients who have previously discontinued paclitaxel-based regimes due to drug related toxicity.', '  Known hypersensitivity to bevacizumab or any component of its formulation', '  Patients who have previously discontinued bevacizumab-containing regimes due to drug-related toxicity.', '  More than three prior lines of chemotherapy', '  Less than three months since last taxane-containing therapy.', '  Wash-out period:', '  Less than three weeks since the last chemotherapy-containing regimen', '  Less than three weeks since the last radiotherapy dose', '  Less than four weeks since last monoclonal antibody-containing therapy', '  Concomitant diseases/conditions:', '  Unstable angina, myocardial infarction, valvular heart disease, encephalopathy, ischemic attacks, hemorrhagic or ischemic cerebrovascular accident (CVA) or ongoing pulmonary embolism within last year, arrhythmia, hepatopathy, uncontrolled infection, hemoptysis or oxygen requiring dyspnea, known HIV infection, bleeding risk, muscular problems, peripheral neuropathy, Symptomatic or progressive brain metastases or leptomeningeal disease.', '  Men or pre-menopausal women who are not using an effective method of contraception as previously described; actively breast feeding women.', '  Patients who have pelvic irradiation with doses  45 Grays (Gy).', '  History of previous bone marrow and/or stem cell transplantation.', '  Confirmed bone marrow involvement']
# Question: does this imply that Patients with a histologically/cytologically confirmed diagnosis of a resectable Non-small cell lung breast cancer or Small cell lung cancer are eligible for the primary trial
# Answer with only one of the options (Entailment or Contradiction):"""
# contradiction

# input_text = """
# Primary trial evidence are INTERVENTION 1:   Laser Therapy Alone therapist administered laser treatment laser: therapist administered laserINTERVENTION 2: Mld Alone therapist administered manual lymphatic drainage manual lymphatic drainage: therapist administered massage therapy Secondary trial evidence are INTERVENTION 1: Part A Abemaciclib: HR+, HER2+ Breast Cancer Abemaciclib 200 mg was administered orally once every 12 hours on days 1-21 of a 21-day cycle when administered as a single agent or in combination with endocrine therapy (ET). Participants with hormone receptor positive HR+, HER2+ breast cancer receiving concurrent trastuzumab, 150 mg abemaciclib was given orally once every 12 hours on days 1-21 of a 21-day cycle. Participants may continue to receive treatment until discontinuation criteria are met.INTERVENTION 2: Part B Abemaciclib: HR+, HER2- Breast Cancer Abemaciclib 200 mg was administered orally once every 12 hours on days 1-21 of a 21-day cycle when administered as a single agent or for in combination with endocrine therapy (ET). Participants may continue to receive treatment until discontinuation criteria are met.
# Question: Does this imply that Laser Therapy is in each cohort of the primary trial and the secondary trial, along with neoadjuvant chemotherapy? Answer with only one of the options (Entailment or Contradiction):
# """
#Contradiction

input_text="""Primary trial evidence are INTERVENTION 1:   Gefitinib (ZD1839)  ZD1839 at a daily dose of 250 mg or 500 mg depending on final dose in parent trial Secondary trial evidence are INTERVENTION 1:   Zoledronic Acid  Zoledronic acid, vitamin D and calcium supplements.INTERVENTION 2:   Zoledronic Acid + Radiopharmaceuticals  Zoledronic acid, vitamin D and calcium supplements, plus Sr-89 or Sm-153.
\n Question: Does this imply that
The intervention in the primary trial consists of a single drug, whereas in the secondary trial the intervention requires at least 3 different drugs?
Answer with only one of the options (Entailment or Contradiction):"""
# Entailment

# input_text="""Primary trial evidence are Inclusion Criteria: Age 52-75 years old; Identification as Latina/Hispanic/Chicana female; Residence in Pilsen, Little Village, East Side or South Chicago; No history of health volunteerism; No history of breast cancer; and Lack of a mammogram within the last two yearsExclusion Criteria: Not meeting all inclusion criteria; Women will be excluded if they participated in formative focus groups."
# \n Question: Does this imply that
# Patients eligible for the primary trial must live in the USA?
# Answer with only one of the options (Entailment or Contradiction):"""
# Entailment


inputs = tokenizer(input_text, return_tensors="pt")

# Forward pass through the model
outputs = model(**inputs)

# Extract logits and compute probabilities
logits = outputs.logits
probs = torch.softmax(logits, dim=-1)

# Get the predicted label
predicted_label = torch.argmax(probs, dim=-1).item()

# Map the label to the actual class
# Assuming the model uses labels 0 = Contradiction, 1 = Entailment
labels = ["Contradiction", "Entailment"]
print(labels[predicted_label])


In [ ]:
#combined_test_zero_shot.csv

import pandas as pd

# File path
file_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/combined_test_zero_shot.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Rename the column
# Replace 'old_column_name' with the current name and 'new_column_name' with the desired name
# df.rename(columns={'model_response': 'mistral-7b-zero-shot'}, inplace=True)
#df
# # Save the updated DataFrame back to the same file
#df.to_csv(file_path, index=False)

# print("Column renamed and file saved successfully!")


In [ ]:
data

# Qwen2-7B

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load Qwen-2-7B model and tokenizer

model_name = "Qwen/Qwen2-7B-Instruct"
# Load model and tokenize
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

base_model_path = "/content/drive/MyDrive/sepi_master_thesis/models/qwen2-original"
model.save_pretrained(base_model_path)
tokenizer.save_pretrained(base_model_path)

In [ ]:
input_text ="""Primary trial evidence are: ['Adverse Events 1:', '  Total: 1/13 (7.69%)', '  Rapid disease progression  [1]1/13 (7.69%)', '  Increased pleural effusion  [2]0/13 (0.00%)', 'Adverse Events 2:', '  Total: 1/4 (25.00%)', '  Rapid disease progression  [1]0/4 (0.00%)', '  Increased pleural effusion  [2]1/4 (25.00%)']
 Question: does this imply that 25% of patients in the primary trial suffer Increased pleural effusion and Rapid disease progression
 Answer with only one of the options (Entailment or Contradiction):"""
 #Contradiction

# input_text="""Primary trial evidence are INTERVENTION 1:   Gefitinib (ZD1839)  ZD1839 at a daily dose of 250 mg or 500 mg depending on final dose in parent trial Secondary trial evidence are INTERVENTION 1:   Zoledronic Acid  Zoledronic acid, vitamin D and calcium supplements.INTERVENTION 2:   Zoledronic Acid + Radiopharmaceuticals  Zoledronic acid, vitamin D and calcium supplements, plus Sr-89 or Sm-153.
# Question: Does this imply that
# The intervention in the primary trial consists of a single drug, whereas in the secondary trial the intervention requires at least 3 different drugs?
# Answer with only one of the options (Entailment or Contradiction):"""
#Entailment

# input_text = """
# Primary trial evidence are INTERVENTION 1:   Laser Therapy Alone therapist administered laser treatment laser: therapist administered laserINTERVENTION 2: Mld Alone therapist administered manual lymphatic drainage manual lymphatic drainage: therapist administered massage therapy Secondary trial evidence are INTERVENTION 1: Part A Abemaciclib: HR+, HER2+ Breast Cancer Abemaciclib 200 mg was administered orally once every 12 hours on days 1-21 of a 21-day cycle when administered as a single agent or in combination with endocrine therapy (ET). Participants with hormone receptor positive HR+, HER2+ breast cancer receiving concurrent trastuzumab, 150 mg abemaciclib was given orally once every 12 hours on days 1-21 of a 21-day cycle. Participants may continue to receive treatment until discontinuation criteria are met.INTERVENTION 2: Part B Abemaciclib: HR+, HER2- Breast Cancer Abemaciclib 200 mg was administered orally once every 12 hours on days 1-21 of a 21-day cycle when administered as a single agent or for in combination with endocrine therapy (ET). Participants may continue to receive treatment until discontinuation criteria are met.
# Question: Does this imply that Laser Therapy is in each cohort of the primary trial and the secondary trial, along with neoadjuvant chemotherapy?
# Answer with only one of the options (Entailment or Contradiction):
# """
#Contradiction

# input_text= """Primary trial evidence are: ['Inclusion Criteria:', '  Voluntarily signed and dated written informed consent', '  Age between 18 and 75 years old (both inclusive)', '  Eastern Cooperative Oncology Group (ECOG) performance status (PS) of  1', '  Life expectancy  3 months.', '  Patients with a histologically/cytologically confirmed diagnosis of advanced and/or unresectable disease of any of the following tumors:', '  Breast cancer', '  Epithelial ovarian cancer or gynecological cancer', '  Head and neck squamous cell carcinoma', '  Non-small cell lung cancer', '  Small cell lung cancer', '  Platinum-refractory germ-cell tumors.', '  Adenocarcinoma or carcinoma of unknown primary site', '  Adequate bone marrow, renal, hepatic, and metabolic function', '  Recovery to grade  1 or to baseline from any Adverse Event (AE) derived from previous treatment (excluding alopecia of any grade).', '  Pre-menopausal women must have a negative pregnancy test before study entry and agree to use a medically acceptable method of contraception throughout the treatment period and for at least six weeks after treatment discontinuation', 'Exclusion Criteria:', '  Prior treatment with PM01183 or weekly paclitaxel or nanoalbumin-paclitaxel', '  Patients who have previously discontinued paclitaxel-based regimes due to drug related toxicity.', '  Known hypersensitivity to bevacizumab or any component of its formulation', '  Patients who have previously discontinued bevacizumab-containing regimes due to drug-related toxicity.', '  More than three prior lines of chemotherapy', '  Less than three months since last taxane-containing therapy.', '  Wash-out period:', '  Less than three weeks since the last chemotherapy-containing regimen', '  Less than three weeks since the last radiotherapy dose', '  Less than four weeks since last monoclonal antibody-containing therapy', '  Concomitant diseases/conditions:', '  Unstable angina, myocardial infarction, valvular heart disease, encephalopathy, ischemic attacks, hemorrhagic or ischemic cerebrovascular accident (CVA) or ongoing pulmonary embolism within last year, arrhythmia, hepatopathy, uncontrolled infection, hemoptysis or oxygen requiring dyspnea, known HIV infection, bleeding risk, muscular problems, peripheral neuropathy, Symptomatic or progressive brain metastases or leptomeningeal disease.', '  Men or pre-menopausal women who are not using an effective method of contraception as previously described; actively breast feeding women.', '  Patients who have pelvic irradiation with doses  45 Grays (Gy).', '  History of previous bone marrow and/or stem cell transplantation.', '  Confirmed bone marrow involvement']
# Question: does this imply that Patients with a histologically/cytologically confirmed diagnosis of a resectable Non-small cell lung breast cancer or Small cell lung cancer are eligible for the primary trial
# Answer with only one of the options (Entailment or Contradiction):"""
# contradiction



# input_text= """Primary trial evidence are Adverse Events 1:  Total: 5/32 (15.63%)  Febrile neutropenia 1/32 (3.13%)  Supraventricular tachycardia 1/32 (3.13%)  Hypersensitivity 2/32 (6.25%)  Catheter site infection 1/32 (3.13%)  Confusional state 1/32 (3.13%) Secondary trial evidence are Adverse Events 1:  Total: 285/752 (37.90%)  Anaemia 2/752 (0.27%)  Disseminated intravascular coagulation 2/752 (0.27%)  Febrile neutropenia 51/752 (6.78%)  Neutropenia 47/752 (6.25%)  Thrombocytopenia 2/752 (0.27%)  Atrial fibrillation 1/752 (0.13%)  Atrial flutter 0/752 (0.00%)  Cardiac failure congestive 1/752 (0.13%)  Left ventricular dysfunction 0/752 (0.00%)Adverse Events 2:  Total: 117/382 (30.63%)  Anaemia 3/382 (0.79%)  Disseminated intravascular coagulation 0/382 (0.00%)  Febrile neutropenia 11/382 (2.88%)  Neutropenia 20/382 (5.24%)  Thrombocytopenia 0/382 (0.00%)  Atrial fibrillation 1/382 (0.26%)  Atrial flutter 1/382 (0.26%)  Cardiac failure congestive 0/382 (0.00%)  Left ventricular dysfunction 1/382 (0.26%) "
# Question: does this imply that Heart-related adverse events were recorded in both the primary trial and the secondary trial.
# Answer with only one of the options (Entailment or Contradiction):"""
# # Entailment



# input_text= """Primary trial evidence are Inclusion Criteria: Age 52-75 years old; Identification as Latina/Hispanic/Chicana female; Residence in Pilsen, Little Village, East Side or South Chicago; No history of health volunteerism; No history of breast cancer; and Lack of a mammogram within the last two yearsExclusion Criteria: Not meeting all inclusion criteria; Women will be excluded if they participated in formative focus groups."
# Question: does this imply that
# Patients eligible for the primary trial must live in the USA.
# Answer with only one of the options (Entailment or Contradiction):"""
#Entailment

# input_text= """Primary trial evidence are Adverse Events 1: Total: 10/71 (14.08%) ATRIAL FIBRILLATION 1/71 (1.41%) CARDIAC TAMPONADE 1/71 (1.41%) PERICARDIAL EFFUSION 1/71 (1.41%) SUPRAVENTRICULAR TACHYCARDIA 1/71 (1.41%) DIARRHOEA 1/71 (1.41%) NAUSEA 1/71 (1.41%) VOMITING 1/71 (1.41%) CHEST PAIN 1/71 (1.41%) PNEUMONIA 1/71 (1.41%) MALIGNANT PLEURAL EFFUSION 1/71 (1.41%) HEPATIC ENCEPHALOPATHY 1/71 (1.41%) .
# Question: does this imply that
# There were only 3 adverse events in the primary trial which occurred more than twice?
# Answer with only one of the options (Entailment or Contradiction):"""
# "Contradiction"
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
outputs = model.generate(inputs['input_ids'], max_length=1024, num_return_sequences=1)

# Decode the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the response
print(response)

In [ ]:
import pandas as pd
from tqdm import tqdm

model = model.to('cuda')

first_five_inputs = df['text'].head(5)

# Initialize progress bar with tqdm
progress_bar = tqdm(enumerate(first_five_inputs, start=1), total=len(first_five_inputs), desc="Processing inputs")

# Loop through the first five inputs and process each one
results = []
for i, input_text in progress_bar:
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to('cuda')
    outputs = model.generate(inputs['input_ids'], max_length=2048, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    results.append(response)
    progress_bar.set_description(f"Processing input {i}")

# Print the results
for i, response in enumerate(results, start=1):
    print(f"Result for input {i}: {response}")


In [ ]:
df.head()
df2=df.head(5)
df2

In [ ]:
import pandas as pd
from tqdm import tqdm  # For progress bar

# Assuming `df` is your DataFrame and 'inputs' is the column containing the text.
text_column = df2['text']

# Initialize progress bar with tqdm
progress_bar = tqdm(enumerate(text_column, start=1), total=len(text_column), desc="Processing inputs")

# Initialize an empty list to store the results
results = []

for i, input_text in progress_bar:
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=4096, padding=True)

    # Generate the output with max_new_tokens=1
    outputs = model.generate(
        inputs['input_ids'],
        max_new_tokens=4,  # Limit output to one word/token
        do_sample=True    # Optional: Set to True for random sampling, or False for deterministic results
    )

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Save the result
    results.append(response.strip())  # Strip extra spaces for cleanliness



In [ ]:
import pandas as pd

# ... your existing code ...

# Create a DataFrame from the results list
results_df = pd.DataFrame({'prediction': results})  # Assuming 'prediction' is a suitable column name

# Save the DataFrame to a CSV file
results_df.to_csv(file_path, index=False)

print("CSV file saved successfully!")

In [ ]:
flan.iloc[7]

In [ ]:
file_path

In [ ]:

import pandas as pd

# File path
file_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/combined_test_zero_shot.csv"
#file_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/responses-zero-shot-qwen.csv"
#file_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/total.csv"
#file_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/responses-two-shot-qwen.csv"
#file_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/flan_t5_xl_2_zero_shot.csv"
#file_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/flan_t5_xl_2.2_zero_shot.csv"
#file_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/flan_t5_response_zero_shot.csv"
#file_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/combined_test_two_shot_mistral.csv"
#file_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/flan_t5_xl__two_shot.csv"
#file_path = "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_03122024/combined_test(text,statement,lables).csv"




## Load the CSV file into a DataFrame
data= pd.read_csv(file_path)
#data2= pd.read_csv(file_path)
#flan= pd.read_csv(file_path)

#all= pd.read_csv(file_path)
#results=pd.read_csv(file_path)
# Rename the column
# Replace 'old_column_name' with the current name and 'new_column_name' with the desired name
# df.rename(columns={'model_response': 'mistral-7b-zero-shot'}, inplace=True)
#df
# # Save the updated DataFrame back to the same file
#df_pred = pd.DataFrame({'prediction': pred})

#data.to_csv(file_path, index=False)
#s.to_csv(file_path, index=False)
#results.to_csv(file_path, index=False)
# print("Column renamed and file saved successfully!")

In [ ]:
data

In [ ]:
# Replace 'prediction' with the name of the column containing the predictions


column_name = 'prediction'

# List to store skipped row numbers
skipped_rows = []

# Iterate through each row and process the column
for index, row in df.iterrows():
    value = row[column_name]
    if value == ['<pad> Yes</s>'] or value == ['<pad> yes</s>']:
        df.at[index, column_name] = 'Entailment'
    elif value == ['<pad> no</s>'] or value == ['<pad> No</s>']:
        df.at[index, column_name] = 'Contradiction'
    elif value == ['<pad> Contradiction</s>']:
        df.at[index, column_name] = 'Contradiction'
    elif value == ['<pad> Entailment</s>']:
        df.at[index, column_name] = 'Entailment'
    elif value not in [['<pad> Entailment</s>'],['<pad> Contradiction</s>']]:
        skipped_rows.append(index)

        #df.at[index, column_name] = None  # Mark for removal later

# Remove rows with None values (skipped rows)
#df = df.dropna(subset=[column_name])

# Save the processed data to a new CSV file


# Output the skipped rows
print("Skipped Rows:", skipped_rows)

## Qwen2 7b finetuning


In [ ]:
# Install all dependencies
!pip install -q transformers accelerate peft datasets bitsandbytes einops
!pip install -q fsspec==2025.3.2
# Imports
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
import torch

In [ ]:
!pip install -U bitsandbytes

In [ ]:
#Download and save  Base Model & Tokenizer (Qwen2-7B-Instruct)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16  # Use float16 if bfloat16 not supported
)

model_id = "Qwen/Qwen2-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)
base_model_path = "/content/drive/MyDrive/sepi_master_thesis/models/qwen2-original"
model.save_pretrained(base_model_path)
tokenizer.save_pretrained(base_model_path)


In [ ]:
#loading the saved model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16  # use float16 for A100
)

# 👇 Load from your saved Google Drive folder
local_model_path = "/content/drive/MyDrive/sepi_master_thesis/models/DeepSeek-R1-Distill-Qwen-7B"

tokenizer = AutoTokenizer.from_pretrained(local_model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

In [ ]:
# Apply QLoRA (PEFT)
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    task_type=TaskType.CAUSAL_LM,
    lora_dropout=0.05,
    bias="none"
)

model = get_peft_model(model, peft_config)

In [ ]:
# dataset
data_files = {
    "train": "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/train-finetune.csv",
    "validation": "/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/dev-finetune.csv"
}

dataset = load_dataset("csv", data_files=data_files)

In [ ]:
#Tokenize Dataset
def tokenize(example):
    result = tokenizer(
        example["text"],  # Make sure "text" is the correct column
        truncation=True,
        padding="max_length",
        max_length=1024
    )

    # Convert label string to single token ID
    label_text = example["label"].strip().lower()
    label_token_id = tokenizer(label_text, add_special_tokens=False)["input_ids"][0]

    result["labels"] = label_token_id
    return result

# ⛏️ Remove original string fields like "label" that crash training
tokenized = dataset.map(tokenize, remove_columns=dataset["train"].column_names)


In [ ]:
def tokenize(example):
    full_input = example["text"].strip() + "\n" + example["label"].strip()
    return tokenizer(full_input, padding="max_length", truncation=True, max_length=1024)

tokenized_datasets = dataset.map(tokenize, batched=True)

In [ ]:
def tokenize(example):
    prompt = example["text"].strip()
    label = example["label"].strip()

    # Full input: prompt + label
    full_input = prompt + "\n" + label

    # Tokenize full sequence
    tokenized = tokenizer(
        full_input,
        padding="max_length",
        truncation=True,
        max_length=1024
    )

    # Tokenize the label only
    label_ids = tokenizer(label, add_special_tokens=False)["input_ids"]

    # Compute position of label tokens
    label_start = len(tokenized["input_ids"]) - len(label_ids)

    # Create labels: mask prompt tokens with -100
    labels = [-100] * label_start + label_ids

    # Ensure labels is 1024 tokens long
    labels = labels[:1024] + [-100] * (1024 - len(labels))

    tokenized["labels"] = labels
    return tokenized
tokenized = dataset.map(tokenize, remove_columns=dataset["train"].column_names)


In [ ]:
#Define Training Arguments
output_path = "/content/drive/MyDrive/sepi_master_thesis/models/DeepSeek-R1-Distill-Qwen-7B-finetuned"


ttraining_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/sepi_master_thesis/models/DeepSeek-R1-Distill-Qwen-7B-finetuned",
    per_device_train_batch_size=4,  # Increased batch size
    per_device_eval_batch_size=4,
    num_train_epochs=5,  # Increased number of epochs
    learning_rate=5e-5,  # Adjusted learning rate
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",


In [ ]:
import os
!pip install wandb
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
#Start Fine-Tuning
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer
)

trainer.train()

In [ ]:
#Save the Fine-Tuned Model
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
import torch

# Load model and tokenizer
local_model_path = "/content/drive/MyDrive/sepi_master_thesis/models/DeepSeek-R1-Distill-Qwen-7B"
tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForCausalLM.from_pretrained(local_model_path)
model.to('cuda')  # Ensure the model is on GPU

# Adjust batch size based on your GPU memory
batch_size = 4  # This is a starting point; adjust based on your GPU

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/sepi_master_thesis/models/DeepSeek-R1-Distill-Qwen-7B-finetuned",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    learning_rate=5e-5,
    fp16=True,  # Enable mixed precision to save memory and potentially speed up training
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    report_to="none"
)

def tokenize(example):
    full_input = example["text"].strip() + "\n" + example["label"].strip()
    return tokenizer(full_input, padding="max_length", truncation=True, max_length=1024)

# Apply tokenization to the dataset
tokenized_datasets = dataset.map(tokenize, batched=True)
# Implement memory cleanup and efficiency
import gc
gc.collect()
torch.cuda.empty_cache()  # Clear any cached memory to avoid OOM

# Load and prepare dataset as previously defined

# Proceed with training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)
trainer.train()


In [ ]:
#Save the Fine-Tuned Model
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)


##  Qwen2 7b finetuning 2


In [ ]:
!pip install -q transformers accelerate peft datasets bitsandbytes einops
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
import pandas as pd
import torch
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"




In [ ]:
# ✅ Load and format your dataset

train_df = pd.read_csv("/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/train-finetune.csv")
dev_df = pd.read_csv("/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/dev-finetune.csv")

def format_fn(example):
    return {
        "text": f"{example['text'].strip()}\nAnswer: {example['label'].strip()}"
    }

train_data = Dataset.from_pandas(train_df).map(format_fn).remove_columns(["label"])
dev_data = Dataset.from_pandas(dev_df).map(format_fn).remove_columns(["label"])

from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen2-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=True  # Save memory
)

# ✅ STEP 4: Tokenize Data
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=1024)

tokenized_train = train_data.map(tokenize, batched=True, remove_columns=["text"])
tokenized_dev = dev_data.map(tokenize, batched=True, remove_columns=["text"])

# ✅ STEP 5: Apply LoRA
from peft import get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

model = get_peft_model(model, peft_config)



In [ ]:
train_data

In [ ]:
# ✅ STEP 6: Define Training Arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/sepi_master_thesis/models/Qwen-7B-finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=5,
    logging_steps=20,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    bf16=True,  # Set to fp16=True if Colab GPU doesn't support bf16
    save_total_limit=1,
    load_best_model_at_end=True,
    report_to="none",
)

# ✅ STEP 7: Define Data Collator and Trainer
from transformers import DataCollatorForLanguageModeling, Trainer

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    data_collator=data_collator,
)

# ✅ STEP 8: Train and Save Model
trainer.train()
trainer.save_model("/content/drive/MyDrive/sepi_master_thesis/models/Qwen-7B-finetuned")


## testing

In [ ]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# ✅ Load your test data
test_df = pd.read_csv("/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/test-zero-shot.csv")  # Make sure it has a 'text' column

# ✅ Load the fine-tuned model
model_path = "/content/drive/MyDrive/sepi_master_thesis/models/Qwen-7B-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True, torch_dtype=torch.float16)

model.eval()

# ✅ Generate predictions
predictions = []

for i, row in test_df.iterrows():
    prompt = row["text"].strip()

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    # Generate prediction
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=8,
            do_sample=False,        # Greedy decoding for consistent results
            eos_token_id=tokenizer.eos_token_id
        )

    # Decode generated text and extract label
    decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract only the answer (after "Answer: ")
    if "Answer:" in decoded:
        predicted_label = decoded.split("Answer:")[-1].strip().split("\n")[0]
    else:
        predicted_label = decoded.strip()

    predictions.append(predicted_label)

# ✅ Save predictions
test_df["predicted_label"] = predictions
#test_df.to_csv("/content/test_predictions.csv", index=False)

print("✅ Predictions saved to /content/test_predictions.csv")
print (test_df)

In [ ]:
test_df.to_csv("/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/test-zero-shot-onQwen2-finetunned.csv", index=False)

# DeepSeek R1


In [ ]:
#!pip install torch transformers datasets accelerate bitsandbytes sentencepiece peft


In [ ]:
!pip install transformers accelerate datasets torch

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer


model_name = "deepseek-ai/deepseek-r1-distill-qwen-7b"
save_directory = "/content/drive/MyDrive/sepi_master_thesis/models/DeepSeek-R1-Distill-Qwen-7B"

# Load tokenizer and model (downloads automatically)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Save model & tokenizer to Google Drive
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

print("✅ Model successfully downloaded and saved in Google Drive!")

In [ ]:
import pandas as pd
from datasets import Dataset


train=pd.read_csv("/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/train-finetune.csv")
dev=pd.read_csv("/content/drive/MyDrive/sepi_master_thesis/new_generated_data_01022025/dev-finetune.csv")

# Convert Pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train)
dev_dataset = Dataset.from_pandas(dev)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "/content/drive/MyDrive/sepi_master_thesis/models/DeepSeek-R1-Distill-Qwen-7B"


tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16,
    device_map="auto"
)
#💡 If you get an A100 or H100 GPU: torch_dtype=torch.bfloat16
#💡 If you get a T4/V100: torch_dtype=torch.float16

print("✅ Model successfully loaded from Google Drive!")

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./deepseek-finetune",
    evaluation_strategy="epoch",  # Evaluate after every epoch
    save_strategy="epoch",  # Save model after each epoch
    per_device_train_batch_size=1,  # Adjust if needed (larger batch → more memory)
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,  # Simulates larger batch size
    learning_rate=2e-5,  # Typical for LLM fine-tuning
    num_train_epochs=2,  # Adjust if needed
    weight_decay=0.01,  # Regularization
    bf16=True,  # Use bfloat16 for lower memory usage
    save_total_limit=2,  # Keep last 2 checkpoints
    logging_dir="./logs",
    logging_steps=50,
    report_to="none"  # No online logging
)

print("Training Parameters Set ✅")

In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,  # Only truncate if text exceeds 1024 tokens
        padding="longest",  # Dynamic padding to match the longest sentence
        max_length=1024  # Increased limit to 1024 tokens
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

trainer.train()

# Start Training
trainer.train()

print("Fine-Tuning Complete ✅")

In [ ]:
model.save_pretrained("/content/drive/MyDrive/sepi_master_thesis/finetuned-deepseek")
tokenizer.save_pretrained("/content/drive/MyDrive/sepi_master_thesis/finetuned-deepseek")

print("Model Saved Successfully ✅")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Path to your fine-tuned model
fine_tuned_model_path = "/content/drive/MyDrive/sepi_master_thesis/models/DeepSeek-R1-Distill-Qwen-7B-finetuned"

# Load tokenizer and model from the fine-tuned directory
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)
model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path)


In [ ]:
data1=data

In [ ]:
data=data1[5:10]

In [ ]:
data

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import torch

# Path to your fine-tuned model
#fine_tuned_model_path = "/content/drive/MyDrive/sepi_master_thesis/models/DeepSeek-R1-Distill-Qwen-7B-finetuned"
fine_tuned_model_path = "/content/drive/MyDrive/sepi_master_thesis/models/qwen2-nli-finetuned"


# Load tokenizer and model from the fine-tuned directory
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)
model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path)

# Move the model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Assuming 'data' is your DataFrame and it has a column named 'text'
responses = []

for input_text in tqdm(data['text'], desc="Processing inputs", total=len(data['text'])):
    # Encode the input text and move tensors to the same device as model
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate output using the model
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,  # Generate a short response
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )

    # Decode and clean up the generated text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    responses.append(response)

# Add responses to the DataFrame
data['model_response'] = responses


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Assuming 'data' is your DataFrame and it has a column named 'text'
responses = []

for input_text in tqdm(data['text'], desc="Processing inputs", total=len(data['text'])):
    # Encode the input text and move tensors to the same device as model
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate output using the model
    outputs = model.generate(
      **inputs,
      max_new_tokens=5,  # Increased length for full reasoning
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,  # Enables sampling for more detailed responses
        temperature=0.6,  # Ensures balanced diversity
        top_p=0.85,  # Helps focus on high-probability tokens
        repetition_penalty=1.1  # Keeps the generation conservative
    )

    # Decode and clean up the generated text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    responses.append(response)

# Add responses to the DataFrame
data['model_response'] = responses

In [ ]:
data